In [11]:
import numpy as np
import pandas as pd
import os
from sklearn.utils import shuffle
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.externals import joblib
from datetime import datetime
import csv
from copy import deepcopy
from sklearn.metrics import mean_squared_error

In [136]:
from lifttable import lifttable
from ks import ks

In [140]:
lifttable=lift(df=df_Purch3to6_test_NL,score='prob_P_3to6mos',DV='PIF_P_3to6mos',Bin=10)

In [141]:
lifttable.to_clipboard()

In [2]:
pd.options.mode.chained_assignment = None

In [19]:
import gc
gc.collect()

7

In [3]:
def lift(df,Bin):
    #df is sorted
    n=int(df.shape[0]/Bin)
    df['bin']=0
    for i in range(Bin):
        a=int(i*n)
        b=int((i+1)*n)
        if i<Bin-1:
            df['bin'][a:b]=i
        else:
            df['bin'][a:]=i

In [17]:
def ks(df, score, DV):
    df.sort_values(score,ascending=False, inplace=True)
    cumgood=(1-df[DV]).cumsum()
    cumbad=df[DV].cumsum()
    pctdiff=abs((cumgood/cumgood.iloc[-1]-cumbad/cumbad.iloc[-1])*100)
    return max(pctdiff)

In [5]:
DataPath=os.path.normpath("S:\Risk Management\Modeling\Projects\Marketing\PurchPropensity\B - Data Exploration\Expandedfinal_v8_zillow_NL.csv")
df_raw=pd.read_csv(DataPath, low_memory=False)
# df=df[df['ActiveListing']==0]
# df.apply(np.random.shuffle, axis=1)
# df['sample_orig']=df['sample']

In [7]:
df.shape

(1070810, 813)

In [15]:
for a in df.columns:
    print("'"+a+"',")

'LoanNumber',
'PropZipChar',
'PropState',
'PropTypeEDW',
'Units',
'OccType',
'OrigDate',
'FirstPmtDate',
'MaturityDate',
'Lien',
'OrigBal',
'OrigVal',
'OrigLTV',
'OrigCLTV',
'ProdType',
'IOFlag',
'IOTerm',
'MaxNegAm',
'PPPFlag',
'PPPTerm',
'OrigTerm',
'OrigRate',
'DTI',
'LoanType',
'LoanPurpose',
'Channel',
'DocType',
'OrigFICO',
'EOM_Date',
'UPB',
'NextDueDate',
'MBA',
'FC',
'BK',
'CurIntRate',
'CurPI',
'Division',
'LossDate',
'LossType',
'L3',
'InvCode',
'Portfolio',
'BoardDateInt',
'PayoffDate',
'PURPOSE',
'BoardDate',
'DropCode_r',
'LPIDate',
'LastModDate',
'LastModDateBom',
'StringPmtsDue',
'ModCntr',
'CurIntRate_r',
'CurPI_r',
'PriorSvcrInfrdMod',
'PmtsDue',
'REO',
'UPBBom',
'PmtsDueBom',
'FCBom',
'BKBom',
'REOBom',
'MBABom',
'PropZipChar1',
'PropZipChar2',
'PropZip',
'CurIntRateBom',
'CurPIBom',
'MosSinceLastBK',
'MosSinceLastBKBom',
'MosInBKSpellBom',
'Exclude',
'ModBooked',
'JumboVintage',
'ModCntrBom',
'MosSinceLastMod',
'SCRA',
'COFI11_l1',
'Prime_l1',
'Libor1m_l1',
'Libor3m

In [7]:
df=df_raw.copy()
df['Prod_ARM']=0
df['Prod_ARM'][df['ProdType'].isin([117,120,124,153])]=1
df['Prod_FIXED']=0
df['Prod_FIXED'][df['ProdType'].isin([110,115,126])]=1
df['Prod_Ballon']=0
df['Prod_Ballon'][df['ProdType']==115]=1
df['Prod_ARM2yr']=0
df['Prod_ARM2yr'][df['ProdType']==124]=1
df['Prod_ARM3yr']=0
df['Prod_ARM3yr'][df['ProdType']==153]=1
df['Prod_ARM5yr']=0
df['Prod_ARM5yr'][df['ProdType']==117]=1
df['Prod_IO1yr']=0
df['Prod_IO1yr'][(df.IOFlag==1) & (0<df['IOTerm']) & (df['IOTerm']<=12)]=1
df['Prod_IO2yr']=0
df['Prod_IO2yr'][(df.IOFlag==1) & (12<df['IOTerm']) & (df['IOTerm']<=24)]=1
df['Prod_IO3yr']=0
df['Prod_IO3yr'][(df.IOFlag==1) & (24<df['IOTerm']) & (df['IOTerm']<=36)]=1
df['Prod_IO5yr']=0
df['Prod_IO5yr'][(df.IOFlag==1) & (36<df['IOTerm']) & (df['IOTerm']<=60)]=1
df['Prod_IO7yr']=0
df['Prod_IO7yr'][(df.IOFlag==1) & (60<df['IOTerm']) & (df['IOTerm']<=84)]=1
df['Prod_IO10yr']=0
df['Prod_IO10yr'][(df.IOFlag==1) & (84<df.IOTerm)]=1

df['LoanType_Conv']=0
df['LoanType_Conv'][df['LoanType'].isin([3,5])]=1
df['LoanType_VA']=0
df['LoanType_VA'][df['LoanType']==4]=1
df['LoanType_FHA']=0
df['LoanType_FHA'][df['LoanType'].isin([1,7,33])]=1
df['LoanType_OthGov']=0
df['LoanType_OthGov'][df['LoanType']==2]=1

df['OrigTermLong']=0
df['OrigTermLong'][df.OrigTerm>365]=1
df['OrigTermShort']=0
df['OrigTermShort'][df.OrigTerm<275]=1

df['NumPifQuote_7d']=df['NumPifQuoteCS_7d']+df['NumPifQuotePD_7d']
df['NumPifQuote_14d']=df['NumPifQuoteCS_14d']+df['NumPifQuotePD_14d']
df['NumPifQuote_28d']=df['NumPifQuoteCS_28d']+df['NumPifQuotePD_28d']

df['StringPmtsDue']=(df['StringPmtsDue'].str.cat(['0'*(24-x) for x in df['StringPmtsDue'].str.len()]))

df['Num30p1to3MosAgo']=((df['StringPmtsDue'].str[1].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[3].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[5].astype(int)>1).astype(int))

df['Num30p4to6MosAgo']=((df['StringPmtsDue'].str[7].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[9].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[11].astype(int)>1).astype(int))

df['Num30p7to9MosAgo']=((df['StringPmtsDue'].str[13].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[15].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[17].astype(int)>1).astype(int))

df['Num30p10to12MosAgo']=((df['StringPmtsDue'].str[19].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[21].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[23].astype(int)>1).astype(int))

df['EOM_Date']=pd.to_datetime(df['EOM_Date'])
df['BoardDate']=pd.to_datetime(df['BoardDate'])
df['AsOfDate']=pd.to_datetime(df['AsOfDate'])

df['MosOnBook']=round((df['EOM_Date']-df['BoardDate'])/np.timedelta64(1, 'M'))

#fillna inplace only works for single columns. Use update for multiple columns
df.update(df[['NumPifQtLast12Mos', 'NumPifQtLast6Mos','NumPifQtLast3Mos', 'NumPifQtLastMo', 
            'AddrMismatch','TextPIF2mosProb', 'TextPIF3to6mosProb','TextPurchase3to6mosProb',
            'TextPIFPurchase3to6mosProb', 'WebPIF2mosProb', 'WebPIF3to6mosProb', 'WebPurchase3to6mosProb',
            'WebPIFPurchase3to6mosProb', 'MaxTextPIF2mosProb_7d', 'MaxTextPIF2mosProb_14d', 'MaxTextPIF2mosProb_28d', 
            'MaxTextPIF3to6mosProb_7d', 'MaxTextPIF3to6mosProb_14d', 'MaxTextPIF3to6mosProb_28d', 'MaxTextP3to6mosProb_7d', 
            'MaxTextP3to6mosProb_14d', 'MaxTextP3to6mosProb_28d', 'MaxTextPIFP3to6mosProb_7d', 'MaxTextPIFP3to6mosProb_14d',
            'MaxTextPIFP3to6mosProb_28d', 'MaxWebPIF2mosProb_7d', 'MaxWebPIF2mosProb_14d', 'MaxWebPIF2mosProb_28d', 
            'MaxWebPIF3to6mosProb_7d', 'MaxWebPIF3to6mosProb_14d', 'MaxWebPIF3to6mosProb_28d', 'MaxWebP3to6mosProb_7d',
            'MaxWebP3to6mosProb_14d', 'MaxWebP3to6mosProb_28d', 'MaxWebPIFP3to6mosProb_7d', 'MaxWebPIFP3to6mosProb_14d',
            'MaxWebPIFP3to6mosProb_28d', 'TotalLogins', 'TotalLogins_7d', 'TotalLogins_14d', 'TotalLogins_28d','NumPifQuote_7d',
            'NumPifQuote_14d', 'NumPifQuote_28d','NumCTLastMo','NumCTLast3Mos','ListedLast3Mos_ind','ListedLast6Mos_ind',
            'ListedLast12Mos_ind','MosSinceLastMod', 'ModCntr', 'BC36']
  ].fillna(0.0))

df['Zillow_nan']=0
df['Zillow_nan'][df.ZillowListingScore.isnull()]=1

# df['FICO_nan']=0
# df['FICO_nan'][df.FICOScore.isnull()]=1

# df['MT34_nan']=0
# df['MT34_nan'][df.MT34.isnull()]=1

# df['S004_nan']=0
# df['S004_nan'][df.S004.isnull()]=1

# df['MT20_nan']=0
# df['MT20_nan'][df.MT20.isnull()]=1

# df['AT99_nan']=0
# df['AT99_nan'][df.AT99.isnull()]=1

# df['BC36_nan']=0
# df['BC36_nan'][df.BC36.isnull()]=1

# df['MT28_nan']=0
# df['MT28_nan'][df.MT28.isnull()]=1

# df['PS253_nan']=0
# df['PS253_nan'][df.PS253.isnull()]=1

# df['MosSinceLastInq']=np.nan
# df['MosSinceLastInq'][df['PS253']>0]=round((pd.to_datetime(df['PS253'][df['PS253']>0].astype(int).apply(str))-
#                                             df['AsOfDate'])/np.timedelta64(1, 'M'))
# df['PS271_nan']=0
# df['PS271_nan'][df.PS271.isnull()]=1

# df['m1mpmnt_nan']=0
# df['m1mpmnt_nan'][df.m1mpmnt.isnull()]=1

# df['m1mthop_nan']=0
# df['m1mthop_nan'][df.m1mthop.isnull()]=1

# df['MosSinLastOpMtg']=np.nan
# df['MosSinLastOpMtg'][df['m1opdte']>0]=round((pd.to_datetime(df['m1opdte'][df['m1opdte']>0].astype(int).apply(str))-
#                                             df['AsOfDate'])/np.timedelta64(1, 'M'))

df['RICETotal7to24Mos_nan']=0
df['RICETotal7to24Mos_nan'][df.RICETotal7to24Mos.isnull()]=1

df['ZillowListingScoreChgMOM']=df['ZillowListingScore']-df['ZillowListingScoreLag1']
df['FICOChgSinceOrig']=df['FICOScore']-df['OrigFICO']

num=['ZillowListingScore', 'ZillowListingScoreChgMOM', 'MTMCLTV', 'RITotal7to24Mos', 'RICETotal7to24Mos','CurPropVal',
    'UnempRt6moChg','UnempRt6moChgPct','UnempRt12moChg','UnempRt12moChgPct','LFPR6moChg','LFPR6moChgPct',
    'LFPR12moChg','LFPR12moChgPct','HHMedInc6moChg','HHMedInc6moChgPct','HHMedInc12moChg','HHMedInc12moChgPct',
    'HHolds6moChg','HHolds6moChgPct','HHolds12moChg','HHolds12moChgPct','HAI6moChg','HAI6moChgPct','HAI12moChg',
    'HAI12moChgPct','MOP6moChg','MOP6moChgPct','MOP12moChg','MOP12moChgPct','MOR6moChg','MOR6moChgPct',
    'MOR12moChg','MOR12moChgPct','SFHS6moChg','SFHS6moChgPct','SFHS12moChg','SFHS12moChgPct','HStarts6moChg',
    'HStarts6moChgPct','HStarts12moChg','HStarts12moChgPct','VacRt6moChg','VacRt6moChgPct','VacRt12moChg',
    'VacRt12moChgPct','Satu90p6moChg','Satu90p6moChgPct','SatuFCL6moChg','SatuFCL6moChgPct','SatuREO6moChg',
    'SatuREO6moChgPct',
#     'FICOScore', 'OrigFICO', 'FICOChgSinceOrig',
#     'AT01', 'AT36', 'RE33', 'RE34', 'MT47', 'MT48', 'G041', 'G042', 'G043', 'G044', 'G083', 'G084', 'G085', 'G094',
#     'G098', 'G102', 'S012', 'S043', 'S059', 'S065', 'AT07', 'AT20', 'AT33', 'MT55', 'BC98', 'G093', 'S064', 'mortcnt',
#     'm1mthop', 'm1terms', 'm1bal', 'm1hicdt', 'm1mpmnt', 'm1cdtlmt', 'AT06', 'AT29', 'AT34', 'AT99', 'BC02',
#     'BC03', 'BC06', 'BC31', 'BC34', 'DS06', 'DS33', 'FI06', 'G057', 'G058', 'G060', 'G062', 'G063', 'G065',
#     'G082', 'G089', 'G091', 'G095', 'G096', 'G104', 'MI21', 'MT01', 'MT02', 'MT03', 'MT20', 'MT21', 'MT22', 'MT28',
#     'MT29', 'MT32', 'MT33', 'MT34', 'MT36', 'MT41', 'MT42', 'MT43', 'MT44', 'MT46', 'MT49', 'MT57', 'PB21', 'PS003',
#     'PS007', 'PS010', 'PS073', 'PS074', 'PS077', 'PS080', 'PS148', 'PS166', 'PS245', 'PS246', 'PS247', 'PS254',
#     'PS256', 'PS257', 'PS265', 'PS266', 'PS267', 'PS268', 'PS269', 'PS270', 'PS271', 'PS272', 'PS273', 'PS274', 'PS275',
#     'S004', 'S060',
#     'MosSinceLastInq'
    ]
df.update(df[num].fillna(df[num].median()))

In [90]:
df=df_raw.copy()
df['Prod_ARM']=0
df['Prod_ARM'][df['ProdType'].isin([117,120,124,153])]=1
df['Prod_FIXED']=0
df['Prod_FIXED'][df['ProdType'].isin([110,115,126])]=1
df['Prod_Ballon']=0
df['Prod_Ballon'][df['ProdType']==115]=1
df['Prod_ARM2yr']=0
df['Prod_ARM2yr'][df['ProdType']==124]=1
df['Prod_ARM3yr']=0
df['Prod_ARM3yr'][df['ProdType']==153]=1
df['Prod_ARM5yr']=0
df['Prod_ARM5yr'][df['ProdType']==117]=1
df['Prod_IO1yr']=0
df['Prod_IO1yr'][(df.IOFlag==1) & (0<df['IOTerm']) & (df['IOTerm']<=12)]=1
df['Prod_IO2yr']=0
df['Prod_IO2yr'][(df.IOFlag==1) & (12<df['IOTerm']) & (df['IOTerm']<=24)]=1
df['Prod_IO3yr']=0
df['Prod_IO3yr'][(df.IOFlag==1) & (24<df['IOTerm']) & (df['IOTerm']<=36)]=1
df['Prod_IO5yr']=0
df['Prod_IO5yr'][(df.IOFlag==1) & (36<df['IOTerm']) & (df['IOTerm']<=60)]=1
df['Prod_IO7yr']=0
df['Prod_IO7yr'][(df.IOFlag==1) & (60<df['IOTerm']) & (df['IOTerm']<=84)]=1
df['Prod_IO10yr']=0
df['Prod_IO10yr'][(df.IOFlag==1) & (84<df.IOTerm)]=1

df['LoanType_Conv']=0
df['LoanType_Conv'][df['LoanType'].isin([3,5])]=1
df['LoanType_VA']=0
df['LoanType_VA'][df['LoanType']==4]=1
df['LoanType_FHA']=0
df['LoanType_FHA'][df['LoanType'].isin([1,7,33])]=1
df['LoanType_OthGov']=0
df['LoanType_OthGov'][df['LoanType']==2]=1

df['OrigTermLong']=0
df['OrigTermLong'][df.OrigTerm>365]=1
df['OrigTermShort']=0
df['OrigTermShort'][df.OrigTerm<275]=1

df['NumPifQuote_7d']=df['NumPifQuoteCS_7d']+df['NumPifQuotePD_7d']
df['NumPifQuote_14d']=df['NumPifQuoteCS_14d']+df['NumPifQuotePD_14d']
df['NumPifQuote_28d']=df['NumPifQuoteCS_28d']+df['NumPifQuotePD_28d']

df['StringPmtsDue']=(df['StringPmtsDue'].str.cat(['0'*(24-x) for x in df['StringPmtsDue'].str.len()]))

df['Num30p1to3MosAgo']=((df['StringPmtsDue'].str[1].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[3].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[5].astype(int)>1).astype(int))

df['Num30p4to6MosAgo']=((df['StringPmtsDue'].str[7].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[9].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[11].astype(int)>1).astype(int))

df['Num30p7to9MosAgo']=((df['StringPmtsDue'].str[13].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[15].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[17].astype(int)>1).astype(int))

df['Num30p10to12MosAgo']=((df['StringPmtsDue'].str[19].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[21].astype(int)>1).astype(int)
                        +(df['StringPmtsDue'].str[23].astype(int)>1).astype(int))

df['EOM_Date']=pd.to_datetime(df['EOM_Date'])
df['BoardDate']=pd.to_datetime(df['BoardDate'])
df['AsOfDate']=pd.to_datetime(df['AsOfDate'])

df['MosOnBook']=round((df['EOM_Date']-df['BoardDate'])/np.timedelta64(1, 'M'))

#fillna inplace only works for single columns. Use update for multiple columns
df.update(df[['NumPifQtLast12Mos', 'NumPifQtLast6Mos','NumPifQtLast3Mos', 'NumPifQtLastMo', 
            'AddrMismatch','TextPIF2mosProb', 'TextPIF3to6mosProb','TextPurchase3to6mosProb',
            'TextPIFPurchase3to6mosProb', 'WebPIF2mosProb', 'WebPIF3to6mosProb', 'WebPurchase3to6mosProb',
            'WebPIFPurchase3to6mosProb', 'MaxTextPIF2mosProb_7d', 'MaxTextPIF2mosProb_14d', 'MaxTextPIF2mosProb_28d', 
            'MaxTextPIF3to6mosProb_7d', 'MaxTextPIF3to6mosProb_14d', 'MaxTextPIF3to6mosProb_28d', 'MaxTextP3to6mosProb_7d', 
            'MaxTextP3to6mosProb_14d', 'MaxTextP3to6mosProb_28d', 'MaxTextPIFP3to6mosProb_7d', 'MaxTextPIFP3to6mosProb_14d',
            'MaxTextPIFP3to6mosProb_28d', 'MaxWebPIF2mosProb_7d', 'MaxWebPIF2mosProb_14d', 'MaxWebPIF2mosProb_28d', 
            'MaxWebPIF3to6mosProb_7d', 'MaxWebPIF3to6mosProb_14d', 'MaxWebPIF3to6mosProb_28d', 'MaxWebP3to6mosProb_7d',
            'MaxWebP3to6mosProb_14d', 'MaxWebP3to6mosProb_28d', 'MaxWebPIFP3to6mosProb_7d', 'MaxWebPIFP3to6mosProb_14d',
            'MaxWebPIFP3to6mosProb_28d', 'TotalLogins', 'TotalLogins_7d', 'TotalLogins_14d', 'TotalLogins_28d','NumPifQuote_7d',
            'NumPifQuote_14d', 'NumPifQuote_28d','NumCTLastMo','NumCTLast3Mos','ListedLast3Mos_ind','ListedLast6Mos_ind',
            'ListedLast12Mos_ind','MosSinceLastMod', 'ModCntr', 'BC36']
  ].fillna(0.0))

df['Zillow_nan']=0
df['Zillow_nan'][df.ZillowListingScore.isnull()]=1

df['FICO_nan']=0
df['FICO_nan'][df.FICOScore.isnull()]=1

df['MT34_nan']=0
df['MT34_nan'][df.MT34.isnull()]=1

df['S004_nan']=0
df['S004_nan'][df.S004.isnull()]=1

df['MT20_nan']=0
df['MT20_nan'][df.MT20.isnull()]=1

df['AT99_nan']=0
df['AT99_nan'][df.AT99.isnull()]=1

df['BC36_nan']=0
df['BC36_nan'][df.BC36.isnull()]=1

df['MT28_nan']=0
df['MT28_nan'][df.MT28.isnull()]=1

df['PS253_nan']=0
df['PS253_nan'][df.PS253.isnull()]=1

df['MosSinceLastInq']=np.nan
df['MosSinceLastInq'][df['PS253']>0]=round((pd.to_datetime(df['PS253'][df['PS253']>0].astype(int).apply(str))-
                                            df['AsOfDate'])/np.timedelta64(1, 'M'))
df['PS271_nan']=0
df['PS271_nan'][df.PS271.isnull()]=1

df['m1mpmnt_nan']=0
df['m1mpmnt_nan'][df.m1mpmnt.isnull()]=1

df['m1mthop_nan']=0
df['m1mthop_nan'][df.m1mthop.isnull()]=1

# df['MosSinLastOpMtg']=np.nan
# df['MosSinLastOpMtg'][df['m1opdte']>0]=round((pd.to_datetime(df['m1opdte'][df['m1opdte']>0].astype(int).apply(str))-
#                                             df['AsOfDate'])/np.timedelta64(1, 'M'))

df['RICETotal7to24Mos_nan']=0
df['RICETotal7to24Mos_nan'][df.RICETotal7to24Mos.isnull()]=1

df['ZillowListingScoreChgMOM']=df['ZillowListingScore']-df['ZillowListingScoreLag1']
df['FICOChgSinceOrig']=df['FICOScore']-df['OrigFICO']

# num=['ZillowListingScore', 'ZillowListingScoreChgMOM', 'MTMCLTV', 'RITotal7to24Mos', 'RICETotal7to24Mos','CurPropVal',
#     'UnempRt6moChg','UnempRt6moChgPct','UnempRt12moChg','UnempRt12moChgPct','LFPR6moChg','LFPR6moChgPct',
#     'LFPR12moChg','LFPR12moChgPct','HHMedInc6moChg','HHMedInc6moChgPct','HHMedInc12moChg','HHMedInc12moChgPct',
#     'HHolds6moChg','HHolds6moChgPct','HHolds12moChg','HHolds12moChgPct','HAI6moChg','HAI6moChgPct','HAI12moChg',
#     'HAI12moChgPct','MOP6moChg','MOP6moChgPct','MOP12moChg','MOP12moChgPct','MOR6moChg','MOR6moChgPct',
#     'MOR12moChg','MOR12moChgPct','SFHS6moChg','SFHS6moChgPct','SFHS12moChg','SFHS12moChgPct','HStarts6moChg',
#     'HStarts6moChgPct','HStarts12moChg','HStarts12moChgPct','VacRt6moChg','VacRt6moChgPct','VacRt12moChg',
#     'VacRt12moChgPct','Satu90p6moChg','Satu90p6moChgPct','SatuFCL6moChg','SatuFCL6moChgPct','SatuREO6moChg',
#     'SatuREO6moChgPct',
#     'FICOScore', 'OrigFICO', 'FICOChgSinceOrig',
#     'AT01', 'AT36', 'RE33', 'RE34', 'MT47', 'MT48', 'G041', 'G042', 'G043', 'G044', 'G083', 'G084', 'G085', 'G094',
#     'G098', 'G102', 'S012', 'S043', 'S059', 'S065', 'AT07', 'AT20', 'AT33', 'MT55', 'BC98', 'G093', 'S064', 'mortcnt',
#     'm1mthop', 'm1terms', 'm1bal', 'm1hicdt', 'm1mpmnt', 'm1cdtlmt', 'AT06', 'AT29', 'AT34', 'AT99', 'BC02',
#     'BC03', 'BC06', 'BC31', 'BC34', 'DS06', 'DS33', 'FI06', 'G057', 'G058', 'G060', 'G062', 'G063', 'G065',
#     'G082', 'G089', 'G091', 'G095', 'G096', 'G104', 'MI21', 'MT01', 'MT02', 'MT03', 'MT20', 'MT21', 'MT22', 'MT28',
#     'MT29', 'MT32', 'MT33', 'MT34', 'MT36', 'MT41', 'MT42', 'MT43', 'MT44', 'MT46', 'MT49', 'MT57', 'PB21', 'PS003',
#     'PS007', 'PS010', 'PS073', 'PS074', 'PS077', 'PS080', 'PS148', 'PS166', 'PS245', 'PS246', 'PS247', 'PS254',
#     'PS256', 'PS257', 'PS265', 'PS266', 'PS267', 'PS268', 'PS269', 'PS270', 'PS271', 'PS272', 'PS273', 'PS274', 'PS275',
#     'S004', 'S060',
#     'MosSinceLastInq', 'MosSinLastOpMtg']
# df.update(df[num].fillna(df[num].median()))

In [8]:
cat_vars=['Portfolio','JumboVintage','OccType','PropState','PropType','Division','Channel','DocType']
dummies=pd.get_dummies(df[cat_vars],dummy_na=True)
df=pd.concat([df,dummies], axis=1)

In [9]:
# LoanNumbers=list(df['LoanNumber'].unique())
# np.random.seed(789)
# R=np.random.rand(len(LoanNumbers))

# VAL=[LoanNumbers[i] for i in range(len(LoanNumbers)) if R[i]<0.2 ]

# df['sample']='DEV'
# df['sample'][df['LoanNumber'].isin(VAL)]='VAL'

df['DV_PIF']=0
df['DV_PIF'][df['PIF_2mos']==1]=1
df['DV_PIF'][df['PIF_3to6mos']==1]=2

df_pif_train_NL=df[(df['sample']=='DEV') & (df['ActiveListing']==0)]
df_pif_test_NL=df[(df['sample']=='VAL') & (df['ActiveListing']==0)]

df_Purch3to6_train_NL=df[(df['sample']=='DEV') & (df['ActiveListing']==0) & (df['PIF_3to6mos']==1) & 
                         ((df['PURPOSE']=='Purchase') | (df['PURPOSE']=='Refi'))]
df_Purch3to6_test_NL=df[(df['sample']=='VAL') & (df['ActiveListing']==0) & (df['PIF_3to6mos']==1) & 
                       ((df['PURPOSE']=='Purchase') | (df['PURPOSE']=='Refi'))]

# Univariate Variable Selection

In [42]:
cat_vars=[]
drop=[]
dtypes=df.dtypes
columns=df.columns
for t in range(len(dtypes)):
    if dtypes[t]=='O' and len(df[columns[t]].unique())>1:
        cat_vars.append(columns[t])
    elif len(df[columns[t]].unique())<=1:
        drop.append(columns[t])

remove=['LoanNumber','PropZipChar','BoardDateInt','PropZipChar1','PropZipChar2','PropZip','StateFip','_TYPE_','ListDate',
        'LastEffDate','LastEndEffDate','PIF_2mos','PIF_3to6mos','CallDate','MBA']+drop
num_vars=[x for x in columns if x not in cat_vars+remove]
remove=['DV_PIF','month', 'DropCode_r', 'EventDate', '_FREQ_','CallDate','EnterDate','AsOfDate','PIF_P_3to6mos',
        'EndEffectiveDate','EndEffectiveDate','Exclude','LPIDate','StringPmtsDue','LastModDateBom','LastModDate',
        'BoardDate','PayoffDate','LossDate','LossType','NextDueDate','EOM_Date','MaturityDate','FirstPmtDate',
        'OrigDate','sample','InvCode','CntyCode','PURPOSE']+drop
cat_vars=[x for x in cat_vars if x not in remove]

In [536]:
df.LoanType.unique()

array([ 1,  5,  3,  4,  2, 28, 29], dtype=int64)

In [43]:
cat_var_PIF_NL=['Portfolio','JumboVintage','OccType','PropState','PropTypeEDW','Division','Channel','DocType']
df_sample=df_pif_train_NL.sample(frac=0.2,axis=0)
dummies=pd.get_dummies(df_sample[cat_vars],dummy_na=True)
# dummies=pd.get_dummies(df_pif_train_NL[cat_var_PIF_NL],dummy_na=True)

Pval_PIF_2mos=list(zip(dummies.columns, chi2(dummies, df_sample['PIF_2mos'])[1]))
Pval_PIF_3to6mos=list(zip(dummies.columns, chi2(dummies, df_sample['PIF_3to6mos'])[1]))

Pval_PIF_2mos.sort(key=lambda t:t[1])
Pval_PIF_3to6mos.sort(key=lambda t:t[1])

In [48]:
Pval_PIF_3to6mos.sort(key=lambda t:t[1])

In [49]:
Pval_PIF_3to6mos[:500]

[('TierCnty_L', 2.623970224476123e-17),
 ('TierMSAD_L', 8.622798302622964e-15),
 ('TierMSA_L', 3.3492354472216136e-14),
 ('TierST_H', 6.415242928151867e-11),
 ('PropType_MFH', 2.353715931569605e-10),
 ('TierCnty_H', 3.8166744859572034e-10),
 ('TierZip_L', 2.832767114156345e-09),
 ('MSACode_OVLL', 3.5036418340599185e-08),
 ('MSACode_OPOW', 1.2256391278545488e-07),
 ('MSACode_OWHI', 4.562354543564441e-07),
 ('TierCnty_M', 6.662546261532598e-07),
 ('MSACode_OSNF', 7.972011959338188e-07),
 ('TierMSA_H', 1.2417930363863987e-06),
 ('MSACode_OSTI', 1.2897186847926415e-06),
 ('TierCnty_nan', 2.5019061194629353e-06),
 ('MSACode_OTHM', 2.082213729638018e-05),
 ('MSACode_OTRU', 3.4273064310888004e-05),
 ('MSACode_OUKI', 7.073906376000287e-05),
 ('TierMSA_nan', 7.512867331703582e-05),
 ('PropType_CONDO', 8.111418537308226e-05),
 ('TierST_L', 0.00015753991427566247),
 ('MSACode_OWIA', 0.00021032075684235927),
 ('PropType_PUD', 0.0002572545584570776),
 ('MSACode_OSTN', 0.0003059212176800308),
 ('MSA

In [28]:
cat_var_PIF_NL=[]
df_sample=df_pif_train_NL.sample(frac=0.1,axis=0)
dummies=pd.get_dummies(df_sample[cat_vars],dummy_na=True)

In [72]:
#Using a samller sample for feature selection
df_sample=df.sample(frac=0.3,axis=0).copy()
# df_sample[num_vars].fillna(df_sample[num_vars].median(), inplace=True)
df_sample=pd.concat([df_sample,pd.get_dummies(df_sample[cat_vars],dummy_na=True)],axis=1)
# df_sample.drop(list(df_select.columns[df_sample.isnull().any()]),inplace=True,axis=1)
df_sample[num_vars].fillna(df_sample[num_vars].median(), inplace=True)

In [78]:
num_vars

['OrigBal',
 'OrigVal',
 'OrigLTV',
 'OrigCLTV',
 'IOFlag',
 'IOTerm',
 'OrigTerm',
 'OrigRate',
 'DTI',
 'LoanType',
 'OrigFICO',
 'UPB',
 'FC',
 'BK',
 'CurIntRate',
 'CurPI',
 'CurIntRate_r',
 'CurPI_r',
 'PriorSvcrInfrdMod',
 'PmtsDue',
 'REO',
 'UPBBom',
 'PmtsDueBom',
 'FCBom',
 'BKBom',
 'REOBom',
 'CurIntRateBom',
 'CurPIBom',
 'MosSinceLastBK',
 'MosSinceLastBKBom',
 'MosInBKSpellBom',
 'ModBooked',
 'MosSinceLastMod',
 'SCRA',
 'COFI11_l1',
 'GT1y_o',
 'GT10y_o',
 'FHLMCFM30_o',
 'FHLMCFM15_o',
 'FHLMCAM1_o',
 'FHLMCAM5_o',
 'CntyFIP',
 'MSADFIP',
 'MSAFIP',
 'LandSqMi',
 'WaterSqMi',
 'Population',
 'Households',
 'ConfLimit1Unit',
 'ConfLimit2Unit',
 'ConfLimit3Unit',
 'ConfLimit4Unit',
 'HPI1Zip_o',
 'HPI1Cnty_o',
 'HPI1MSAD_o',
 'HPI1MSA_o',
 'HPI1ST_o',
 'HPI1LZip_o',
 'HPI1LCnty_o',
 'HPI1LMSAD_o',
 'HPI1LMSA_o',
 'HPI1LST_o',
 'HPI1MZip_o',
 'HPI1MCnty_o',
 'HPI1MMSAD_o',
 'HPI1MMSA_o',
 'HPI1MST_o',
 'HPI1HZip_o',
 'HPI1HCnty_o',
 'HPI1HMSAD_o',
 'HPI1HMSA_o',
 'HPI1H

In [80]:
list(df_sample.columns[df_sample.isnull().any()])

['OrigLTV',
 'OrigCLTV',
 'MaxNegAm',
 'PPPFlag',
 'PPPTerm',
 'DTI',
 'DocType',
 'OrigFICO',
 'NextDueDate',
 'MBA',
 'Division',
 'LossDate',
 'LossType',
 'L3',
 'InvCode',
 'Portfolio',
 'PURPOSE',
 'LPIDate',
 'LastModDate',
 'LastModDateBom',
 'PmtsDue',
 'MosSinceLastBK',
 'MosSinceLastBKBom',
 'MosInBKSpellBom',
 'MosSinceLastMod',
 'CntyFIP',
 'CntyCode',
 'MSADCode',
 'MSADFIP',
 'MSACode',
 'MSAFIP',
 'LandSqMi',
 'WaterSqMi',
 'Population',
 'Households',
 'ConfLimit1Unit',
 'ConfLimit2Unit',
 'ConfLimit3Unit',
 'ConfLimit4Unit',
 'HPI1Zip_o',
 'HPI1Cnty_o',
 'HPI1MSAD_o',
 'HPI1MSA_o',
 'HPI1ST_o',
 'HPI1LZip_o',
 'HPI1LCnty_o',
 'HPI1LMSAD_o',
 'HPI1LMSA_o',
 'HPI1LST_o',
 'HPI1MZip_o',
 'HPI1MCnty_o',
 'HPI1MMSAD_o',
 'HPI1MMSA_o',
 'HPI1MST_o',
 'HPI1HZip_o',
 'HPI1HCnty_o',
 'HPI1HMSAD_o',
 'HPI1HMSA_o',
 'HPI1HST_o',
 'TB1LMZip_o',
 'TB1LMCnty_o',
 'TB1LMMSAD_o',
 'TB1LMMSA_o',
 'TB1LMST_o',
 'TB1MHZip_o',
 'TB1MHCnty_o',
 'TB1MHMSAD_o',
 'TB1MHMSA_o',
 'TB1MHST_o',


# Step 1 - PIF Model

In [27]:
input_features=list(set(
    ['AddrMismatch',
    'MaxTextPIF2mosProb_7d', 'MaxTextPIF2mosProb_14d', 'MaxTextPIF2mosProb_28d', 
    'MaxTextPIF3to6mosProb_7d', 'MaxTextPIF3to6mosProb_14d', 'MaxTextPIF3to6mosProb_28d', 'MaxTextP3to6mosProb_7d', 
    'MaxTextP3to6mosProb_14d', 'MaxTextP3to6mosProb_28d', 'MaxTextPIFP3to6mosProb_7d', 'MaxTextPIFP3to6mosProb_14d',
    'MaxTextPIFP3to6mosProb_28d', 'MaxWebPIF2mosProb_7d', 'MaxWebPIF2mosProb_14d', 'MaxWebPIF2mosProb_28d', 
    'MaxWebPIF3to6mosProb_7d', 'MaxWebPIF3to6mosProb_14d', 'MaxWebPIF3to6mosProb_28d', 'MaxWebP3to6mosProb_7d',
    'MaxWebP3to6mosProb_14d', 'MaxWebP3to6mosProb_28d', 'MaxWebPIFP3to6mosProb_7d', 'MaxWebPIFP3to6mosProb_14d',
    'MaxWebPIFP3to6mosProb_28d',
    'TotalLogins', 'TotalLogins_7d', 'TotalLogins_14d', 'TotalLogins_28d',
    'NumPifQuote_7d','NumPifQuote_14d', 'NumPifQuote_28d', 'NumPifQtLast12Mos', 'NumPifQtLast6Mos',
    'NumPifQtLast3Mos', 'NumPifQtLastMo',
    'Prod_ARM','Prod_FIXED','Prod_Ballon','Prod_ARM2yr','Prod_ARM3yr', 'Prod_ARM5yr','Prod_IO1yr','Prod_IO2yr','Prod_IO3yr',
    'Prod_IO5yr','Prod_IO7yr','Prod_IO10yr',
    'OrigTermLong','OrigTermShort','CurIntRate','UPB','SATO','OrigVal','OrigCLTV','MTMCLTV','LoanAge',
    'RIMaxLast6mos','RIMaxLast12mos','RIMaxLast24mos','RICEMaxLast6mos','RICEMaxLast12mos','RICEMaxLast24mos',
    'RITotal7to24Mos','RICETotal7to24Mos','RIMax7to24mos','RICEMax7to24mos',
    'Num30p1to3MosAgo','Num30p4to6MosAgo','Num30p7to9MosAgo','Num30p10to12MosAgo','Num30pLast12Mos',
    'CurPropVal','MosSinceLastMod', 'ModCntr', 'HARPElig', 'IOFlag','Jumbo',
    'MosOnBook','Zillow_nan',
    'NumCTLastMo','NumCTLast3Mos','ListedLast3Mos_ind','ListedLast6Mos_ind','ListedLast12Mos_ind'
    ]+list(dummies.columns)+num))

In [118]:
num_vars

['Units',
 'Lien',
 'OrigBal',
 'OrigVal',
 'OrigLTV',
 'OrigCLTV',
 'ProdType',
 'IOFlag',
 'IOTerm',
 'MaxNegAm',
 'PPPFlag',
 'PPPTerm',
 'OrigTerm',
 'OrigRate',
 'DTI',
 'LoanType',
 'OrigFICO',
 'UPB',
 'FC',
 'BK',
 'CurIntRate',
 'CurPI',
 'ModCntr',
 'CurIntRate_r',
 'CurPI_r',
 'PriorSvcrInfrdMod',
 'PmtsDue',
 'REO',
 'UPBBom',
 'PmtsDueBom',
 'FCBom',
 'BKBom',
 'REOBom',
 'MBABom',
 'CurIntRateBom',
 'CurPIBom',
 'MosSinceLastBK',
 'MosInBKSpellBom',
 'ModBooked',
 'ModCntrBom',
 'MosSinceLastMod',
 'SCRA',
 'COFI11_l1',
 'Prime_l1',
 'Libor1m_l1',
 'Libor3m_l1',
 'Libor6m_l1',
 'Libor12m_l1',
 'TB3m_l1',
 'TB6m_l1',
 'GT1y_l1',
 'GT3y_l1',
 'GT5y_l1',
 'GT10y_l1',
 'FHFBE_l1',
 'CD6M_l1',
 'FHLMCFM30_l1',
 'FHLMCFM15_l1',
 'FHLMCAM1_l1',
 'FHLMCAM5_l1',
 'GT1y_o',
 'GT10y_o',
 'FHLMCFM30_o',
 'FHLMCFM15_o',
 'FHLMCAM1_o',
 'FHLMCAM5_o',
 'CntyFIP',
 'MSADFIP',
 'MSAFIP',
 'LandSqMi',
 'WaterSqMi',
 'Population',
 'Households',
 'ConfLimit1Unit',
 'ConfLimit2Unit',
 'ConfL

In [59]:
list(df_pif_train_NL[input_features].columns[df_pif_train_NL[input_features].isnull().any()])

[]

In [37]:
df_pif_train_sample.shape

(751039, 967)

In [ ]:
Pval_PIF_2mos=list(zip(df_select.columns, chi2(df_select, df['PIF_2mos'])[1]))
Pval_PIF_3to6mos=list(zip(df_select.columns, chi2(df_select, df['PIF_3to6mos'])[1]))

Pval_PIF_2mos.sort(key=lambda t:t[1])
Pval_PIF_3to6mos.sort(key=lambda t:t[1])

## Gradient Boosting

In [40]:
df_pif_train_sample=df_pif_train_NL.sample(frac=0.1, random_state=456)

In [120]:
# for n in [300,400,500]:
n=300
depth=4
lr=0.05
# for lr in [0.05,0.1]:
#     for depth in [3,6,9]:
gbclf_PIF_NL=GradientBoostingClassifier(loss='deviance', n_estimators=n, max_depth=depth, subsample=0.1, random_state=456,
                                        verbose=1, learning_rate=lr)
gbclf_PIF_NL.fit(df_pif_train_NL[PIF_input_features],df_pif_train_NL['DV_PIF'])
# gbclf_PIF_NL.fit(df_pif_train_sample[input_features],df_pif_train_sample['DV_PIF'])
print('n_estimators='+str(n))
print('Learning Rate='+str(lr))
print('Max_depth='+str(depth))
print(gbclf_PIF_NL.score(df_pif_train_NL[PIF_input_features],df_pif_train_NL['DV_PIF']))
print(gbclf_PIF_NL.score(df_pif_test_NL[PIF_input_features],df_pif_test_NL['DV_PIF']))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1      149096.1085       80630.3644          403.15m
         2      140528.0532       73603.2218          384.94m
         3      133358.2682       67011.2024          377.21m
         4      126415.6542       61315.5716          374.88m
         5      120348.8312       56012.7412          380.10m
         6      114678.1432       51230.2438          381.88m
         7      109354.6021       47006.3888          381.68m
         8      104456.3845       43177.6536          379.29m
         9      100125.2758       39599.6239          386.56m
        10       96081.3572       36330.6898          392.61m
        20       69671.4417       15719.6363          372.49m
        30       59103.0090        6653.3642          356.96m
        40       54096.2565        2922.3284          349.93m
        50       51736.3682        1416.2985          337.95m
        60       50060.9258         770.2578          322.67m
       

In [ ]:
0.9537192895278618
0.9516758341816008

In [122]:
len(gbclf_PIF_NL.feature_importances_)

163

In [123]:
importances=zip(gbclf_PIF_NL.feature_importances_,PIF_input_features)
ListImp=list(importances)
sorted(ListImp)

[(0.0001006907353560137, 'Prod_ARM2yr'),
 (0.00010838698247351635, 'Channel_Retail'),
 (0.00011316008939194679, 'PropState_AL'),
 (0.00011394299193034064, 'Jumbo'),
 (0.00016475997250092633, 'PropState_RI'),
 (0.00016564463572727096, 'PropState_SC'),
 (0.00018865839227038452, 'PropState_SD'),
 (0.0002510531711221297, 'PropState_TN'),
 (0.0002702875185790084, 'PropState_TX'),
 (0.0002863223976116868, 'Prod_ARM'),
 (0.0002900835393289428, 'PropState_OR'),
 (0.0003288936295830819, 'PropState_IL'),
 (0.0003365331579004705, 'Prod_IO7yr'),
 (0.0003947942255655531, 'PropState_CT'),
 (0.0004137552573375465, 'PropState_DC'),
 (0.00041810942742505526, 'Channel_Direct'),
 (0.0004367061607777088, 'PropState_MN'),
 (0.000438379552109974, 'Division_ABS'),
 (0.00047704317388640134, 'JumboVintage_2015'),
 (0.0004902264386074077, 'PropState_PA'),
 (0.0004919208083806969, 'PropState_MD'),
 (0.0005269153864136954, 'NumPifQtLastMo'),
 (0.0005820869349706121, 'PropState_KS'),
 (0.0005874696425316806, 'Prop

In [121]:
len(PIF_input_features)

163

In [63]:
drop=[x[1] for x in ListImp if x[0]<1e-4]
input_features=[x for x in input_features if x not in drop]

In [53]:
input_features=input_features+['FICO_nan', 'MT34_nan', 'S004_nan', 'MT20_nan', 'AT99_nan', 'm1mpmnt_nan', 'BC36_nan', 
                               'PS271_nan', 'MT28_nan', 'PS253_nan', 'MosSinceLastInq']

In [58]:
input_features.remove('m1opdte')

In [124]:
with open('out/PIF_input_features_woTU.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(PIF_input_features)

In [13]:
with open('out/PIF_input_features.csv', newline='') as myfile:
    reader=csv.reader(myfile)
    input_features=list()
    for row in reader:
        input_features=input_features+row

In [100]:
TUData=['FICO_nan', 'MT34_nan', 'S004_nan', 'MT20_nan', 'AT99_nan', 'm1mpmnt_nan', 'BC36_nan', 
                               'PS271_nan', 'MT28_nan', 'PS253_nan', 'MosSinceLastInq',
           'FICOScore', 'OrigFICO', 'FICOChgSinceOrig',
    'AT01', 'AT36', 'BC36', 'RE33', 'RE34', 'MT47', 'MT48', 'G041', 'G042', 'G043', 'G044', 'G083', 'G084', 'G085', 'G094',
    'G098', 'G102', 'S012', 'S043', 'S059', 'S065', 'AT07', 'AT20', 'AT33', 'MT55', 'BC98', 'G093', 'S064', 'mortcnt',
    'm1mthop', 'm1terms', 'm1bal', 'm1hicdt', 'm1mpmnt', 'm1cdtlmt', 'AT06', 'AT29', 'AT34', 'AT99', 'BC02',
    'BC03', 'BC06', 'BC31', 'BC34', 'DS06', 'DS33', 'FI06', 'G057', 'G058', 'G060', 'G062', 'G063', 'G065',
    'G082', 'G089', 'G091', 'G095', 'G096', 'G104', 'MI21', 'MT01', 'MT02', 'MT03', 'MT20', 'MT21', 'MT22', 'MT28',
    'MT29', 'MT32', 'MT33', 'MT34', 'MT36', 'MT41', 'MT42', 'MT43', 'MT44', 'MT46', 'MT49', 'MT57', 'PB21', 'PS003',
    'PS007', 'PS010', 'PS073', 'PS074', 'PS077', 'PS080', 'PS148', 'PS166', 'PS245', 'PS246', 'PS247', 'PS254',
    'PS256', 'PS257', 'PS265', 'PS266', 'PS267', 'PS268', 'PS269', 'PS270', 'PS271', 'PS272', 'PS273', 'PS274', 'PS275',
    'S004', 'S060', 'm1lntyp_FR', 'm1lntyp_VM', 'm1lntyp_CV', 'm1lntyp_SE', 'm1lntyp_CY', 'm1lntyp_SM']

In [101]:
PIF_input_features=[x for x in input_features if x not in TUData]

In [102]:
PIF_input_features

['Division_Freddie',
 'UnempRt12moChg',
 'MaxTextPIF2mosProb_28d',
 'PropState_OR',
 'MaxTextPIFP3to6mosProb_14d',
 'SatuFCL6moChgPct',
 'HAI6moChg',
 'SatuFCL6moChg',
 'MOP12moChgPct',
 'LFPR12moChg',
 'PropState_NH',
 'LoanAge',
 'MaxTextP3to6mosProb_28d',
 'NumPifQtLast12Mos',
 'HStarts6moChg',
 'MOP12moChg',
 'Prod_IO7yr',
 'AddrMismatch',
 'Prod_ARM3yr',
 'HHolds12moChg',
 'PropState_MI',
 'MaxWebPIF3to6mosProb_14d',
 'PropState_UT',
 'HHolds12moChgPct',
 'UPB',
 'MaxWebP3to6mosProb_7d',
 'UnempRt12moChgPct',
 'HHolds6moChg',
 'PropState_TX',
 'PropState_SD',
 'PropState_MN',
 'RICETotal7to24Mos',
 'OrigCLTV',
 'MosSinceLastMod',
 'PropState_ID',
 'MOP6moChg',
 'PropState_CT',
 'PropState_MT',
 'VacRt12moChgPct',
 'NumPifQuote_7d',
 'Portfolio_PINE (FHLMC)',
 'PropState_DC',
 'RICEMaxLast24mos',
 'PropType_SFH',
 'MaxTextPIFP3to6mosProb_7d',
 'Satu90p6moChg',
 'MOP6moChgPct',
 'Prod_ARM5yr',
 'MosOnBook',
 'Prod_ARM2yr',
 'MOR6moChg',
 'LFPR6moChg',
 'Zillow_nan',
 'Num30pLast12Mo

In [125]:
joblib.dump(gbclf_PIF_NL, 'out/PIF_GB_woTU.pkl')

['out/PIF_GB_woTU.pkl']

## Random Forest

In [424]:
###################
# No Zillow Score
###################

input_features.remove('ZillowListingScore')

# for depth in [18,20,22]:
# for samples in [500,1000]:
# for n in [400,500,600]:
rfclf_PIF_NL=RandomForestClassifier(n_estimators=300, max_depth=15, min_samples_split=2000,
                                    random_state=159, n_jobs=16, verbose=0)
# rfclf_PIF_NL=GradientBoostingClassifier(loss='deviance', n_estimators=100, max_depth=6, subsample=0.3, 
#                                         random_state=123, verbose=1, learning_rate=0.1)
rfclf_PIF_NL.fit(df_pif_train_NL[input_features],df_pif_train_NL['DV_PIF'])
# print('Max_depth='+str(depth))
# print('Learning Rate='+str(lr))
# print('max_depth='+str(depth)+' min_samples_split='+str(samples))
print(rfclf_PIF_NL.score(df_pif_train_NL[input_features],df_pif_train_NL['DV_PIF']))
print(rfclf_PIF_NL.score(df_pif_test_NL[input_features],df_pif_test_NL['DV_PIF']))

0.5657224619129329
0.5182040626439841


In [512]:
# for samples in [500,1000]:
# for n in [100,200,300,400,500]:
# for depth in [35,]:
rfclf_PIF_NL=RandomForestClassifier(n_estimators=100, max_depth=depth, min_samples_split=1000,
                                    random_state=159, n_jobs=16, verbose=0)
# rfclf_PIF_NL=GradientBoostingClassifier(loss='deviance', n_estimators=100, max_depth=6, subsample=0.3, 
#                                         random_state=123, verbose=1, learning_rate=0.1)
rfclf_PIF_NL.fit(df_pif_train_NL[input_features],df_pif_train_NL['DV_PIF'])
#     print('Max_depth='+str(n))
print('Learning Rate='+str(depth))
# print('max_depth='+str(depth)+' min_samples_split='+str(samples))
print(rfclf_PIF_NL.score(df_pif_train_NL[input_features],df_pif_train_NL['DV_PIF']))
print(rfclf_PIF_NL.score(df_pif_test_NL[input_features],df_pif_test_NL['DV_PIF']))

0.9535886216313069
0.9537432991197506


In [508]:
importances=zip(rfclf_PIF_NL.feature_importances_,input_features)
ListImp=list(importances)
sorted(ListImp)

[(0.00012261813457255916, 'PropTypeEDW_Single Family Row'),
 (0.00012919778045822607, 'Prod_Ballon'),
 (0.00013383893444321394, 'OccType_Unknown'),
 (0.00018394434839758383, 'Portfolio_nan'),
 (0.00019904423876592882, 'TotalLogins'),
 (0.00021008669512311018, 'PropState_SD'),
 (0.00024279926036283029, 'PropTypeEDW_Single Family Attached'),
 (0.00025374276650490127, 'Prod_IO7yr'),
 (0.0002646598438614668, 'Prod_IO10yr'),
 (0.00027020528108300456, 'Division_Special'),
 (0.0002702133322766566, 'PropState_WV'),
 (0.00027621428177824066, 'DocType_nan'),
 (0.00035974291896567577, 'DocType_Full'),
 (0.0003752329650485846, 'PropState_CT'),
 (0.0003847082533417516, 'Prod_ARM2yr'),
 (0.0003881066769353019, 'Portfolio_Aurora (Freddie)'),
 (0.00039285458767729527, 'DocType_LID'),
 (0.00041546118690263735, 'PropState_DC'),
 (0.0004226502672797106, 'Channel_Bulk Acquisition'),
 (0.0004529320948845463, 'JumboVintage_2015'),
 (0.00047341545428988835, 'Channel_Retail'),
 (0.0004736865790787999, 'Portfo

In [557]:
for a in sorted(ListImp):
    print(a[1])

PropTypeEDW_Single Family Row
Prod_Ballon
OccType_Unknown
Portfolio_nan
TotalLogins
PropState_SD
PropTypeEDW_Single Family Attached
Prod_IO7yr
Prod_IO10yr
Division_Special
PropState_WV
DocType_nan
DocType_Full
PropState_CT
Prod_ARM2yr
Portfolio_Aurora (Freddie)
DocType_LID
PropState_DC
Channel_Bulk Acquisition
JumboVintage_2015
Channel_Retail
Portfolio_Securities/SFO
JumboVintage_2010
Portfolio_HSBC (Freddie)
Prod_ARM3yr
PropState_AK
PropTypeEDW_Mobile Home
JumboVintage_2014
PropTypeEDW_No Appraisal on file
DocType_NIV
PropState_NJ
PropTypeEDW_Duplex
Prod_IO5yr
Division_ABS
PropState_VT
PropTypeEDW_Quadruplex
PropTypeEDW_Manufactured Home
PropState_NH
PropState_MS
Portfolio_PINE (FHLMC)
PropState_OK
PropState_MT
Portfolio_FNMA-NSTR
MaxTextPIFP3to6mosProb_7d
PropState_WY
Channel_Wholesale
PropState_NC
JumboVintage_<2010
Channel_Direct
PropState_FL
MaxTextPIF3to6mosProb_7d
PropState_NM
Portfolio_NSM (Freddie)
Portfolio_Inventory
PropState_MD
PropState_OH
NumPifQtLastMo
PropTypeEDW_Triple

In [510]:
input_features=input_features+drop

In [503]:
drop=[x[1] for x in ListImp if x[0]<1e-4]
input_features=[x for x in input_features if x not in drop]

In [511]:
len(input_features)

169

In [513]:
pred=rfclf_PIF_NL.predict_proba(df_pif_test_NL[input_features])
df_pif_test_NL['prob_2mos']=pred[:,1]
df_pif_test_NL['prob_3to6mos']=pred[:,2]
score=df_pif_test_NL[['prob_2mos','prob_3to6mos','PIF_3to6mos','PIF_2mos']].copy()

In [515]:
score.sort_values('prob_2mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_2mos','bin']].groupby('bin').agg(['count','sum','mean'])

PIF_2mos                
       count   sum      mean
bin                         
0      63629  4316  0.067831
1      63629   708  0.011127
2      63629   509  0.007999
3      63629   395  0.006208
4      63629   374  0.005878
5      63629   432  0.006789
6      63629   331  0.005202
7      63629   249  0.003913
8      63629   142  0.002232
9      63636   115  0.001807

In [514]:
score.sort_values('prob_3to6mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

PIF_3to6mos                
          count   sum      mean
bin                            
0         63629  5516  0.086690
1         63629  3054  0.047997
2         63629  2581  0.040563
3         63629  2292  0.036021
4         63629  2112  0.033192
5         63629  2134  0.033538
6         63629  1615  0.025382
7         63629  1767  0.027770
8         63629  1298  0.020400
9         63636   799  0.012556

In [504]:
score.sort_values('prob_3to6mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

PIF_3to6mos                
          count   sum      mean
bin                            
0         63629  5493  0.086329
1         63629  2979  0.046818
2         63629  2549  0.040060
3         63629  2583  0.040595
4         63629  2051  0.032234
5         63629  1938  0.030458
6         63629  1747  0.027456
7         63629  1573  0.024721
8         63629  1382  0.021720
9         63636   873  0.013719

In [518]:
# input_features=list(zip([feature_names[i] for i in PR_train_text_index], PR_train_text_index))
with open('rfclf_PIF_NL_input_features.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(input_features)

In [519]:
joblib.dump(rfclf_PIF_NL, 'rfclf_PIF_NL.pkl')

['rfclf_PIF_NL.pkl']

# Step 2 - Purchase Model

## Gradient Boosting

In [64]:
input_features=list(set(
    ['AddrMismatch',
    'MaxTextPIF2mosProb_7d', 'MaxTextPIF2mosProb_14d', 'MaxTextPIF2mosProb_28d', 
    'MaxTextPIF3to6mosProb_7d', 'MaxTextPIF3to6mosProb_14d', 'MaxTextPIF3to6mosProb_28d', 'MaxTextP3to6mosProb_7d', 
    'MaxTextP3to6mosProb_14d', 'MaxTextP3to6mosProb_28d', 'MaxTextPIFP3to6mosProb_7d', 'MaxTextPIFP3to6mosProb_14d',
    'MaxTextPIFP3to6mosProb_28d', 'MaxWebPIF2mosProb_7d', 'MaxWebPIF2mosProb_14d', 'MaxWebPIF2mosProb_28d', 
    'MaxWebPIF3to6mosProb_7d', 'MaxWebPIF3to6mosProb_14d', 'MaxWebPIF3to6mosProb_28d', 'MaxWebP3to6mosProb_7d',
    'MaxWebP3to6mosProb_14d', 'MaxWebP3to6mosProb_28d', 'MaxWebPIFP3to6mosProb_7d', 'MaxWebPIFP3to6mosProb_14d',
    'MaxWebPIFP3to6mosProb_28d',
    'TotalLogins', 'TotalLogins_7d', 'TotalLogins_14d', 'TotalLogins_28d',
    'NumPifQuote_7d','NumPifQuote_14d', 'NumPifQuote_28d', 'NumPifQtLast12Mos', 'NumPifQtLast6Mos',
    'NumPifQtLast3Mos', 'NumPifQtLastMo',
    'Prod_ARM','Prod_FIXED','Prod_Ballon','Prod_ARM2yr','Prod_ARM3yr', 'Prod_ARM5yr','Prod_IO1yr','Prod_IO2yr','Prod_IO3yr',
    'Prod_IO5yr','Prod_IO7yr','Prod_IO10yr',
    'OrigTermLong','OrigTermShort','CurIntRate','UPB','SATO','OrigVal','OrigCLTV','MTMCLTV','LoanAge',
    'RIMaxLast6mos','RIMaxLast12mos','RIMaxLast24mos','RICEMaxLast6mos','RICEMaxLast12mos','RICEMaxLast24mos',
    'RITotal7to24Mos','RICETotal7to24Mos','RIMax7to24mos','RICEMax7to24mos',
    'Num30p1to3MosAgo','Num30p4to6MosAgo','Num30p7to9MosAgo','Num30p10to12MosAgo','Num30pLast12Mos',
    'CurPropVal','MosSinceLastMod', 'ModCntr', 'HARPElig', 'IOFlag','Jumbo',
    'MosOnBook','Zillow_nan',
    'NumCTLastMo','NumCTLast3Mos','ListedLast3Mos_ind','ListedLast6Mos_ind','ListedLast12Mos_ind'
    ]+list(dummies.columns)+num+['FICO_nan', 'MT34_nan', 'S004_nan', 'MT20_nan', 'AT99_nan', 'm1mpmnt_nan', 'BC36_nan', 
                               'PS271_nan', 'MT28_nan', 'PS253_nan', 'RICETotal7to24Mos_nan'])
                   )

In [116]:
#for loss in ['deviance', 'exponential']:
loss='exponential'
n=500
depth=4
lr=0.03
gbclf_P_NL=GradientBoostingClassifier(loss=loss, n_estimators=n, max_depth=depth, subsample=0.5, random_state=789,
                                    verbose=1, learning_rate=lr)
gbclf_P_NL.fit(df_Purch3to6_train_NL[Purchase_input_features],df_Purch3to6_train_NL['PIF_P_3to6mos'])
print('n_estimators='+str(n))
print('Learning Rate='+str(lr))
print('Max_depth='+str(depth))
print(gbclf_P_NL.score(df_Purch3to6_train_NL[Purchase_input_features],df_Purch3to6_train_NL['PIF_P_3to6mos']))
print(gbclf_P_NL.score(df_Purch3to6_test_NL[Purchase_input_features],df_Purch3to6_test_NL['PIF_P_3to6mos']))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.9920           0.0050            5.35m
         2           0.9872           0.0049            4.95m
         3           0.9825           0.0047            4.76m
         4           0.9779           0.0043            4.67m
         5           0.9740           0.0041            4.64m
         6           0.9694           0.0040            4.73m
         7           0.9652           0.0037            4.67m
         8           0.9619           0.0037            4.68m
         9           0.9584           0.0036            4.71m
        10           0.9551           0.0033            4.68m
        20           0.9269           0.0023            4.76m
        30           0.9082           0.0017            4.65m
        40           0.8925           0.0012            4.51m
        50           0.8800           0.0010            4.47m
        60           0.8713           0.0006            4.34m
       

In [117]:
importances=zip(gbclf_P_NL.feature_importances_,Purchase_input_features)
ListImp=list(importances)
sorted(ListImp)

[(0.00010877512043859741, 'Channel_Wholesale'),
 (0.00012000765301565334, 'MaxTextPIFP3to6mosProb_7d'),
 (0.00021333820682646634, 'Prod_ARM2yr'),
 (0.00024894565363439903, 'MaxWebPIF2mosProb_7d'),
 (0.00027418970417432084, 'Division_ABS'),
 (0.00030931505814146674, 'PropState_VA'),
 (0.00031109869518387633, 'PropState_FL'),
 (0.00034924167843958725, 'NumPifQuote_7d'),
 (0.00035334424233843145, 'PropState_NY'),
 (0.00036786786464006204, 'PropState_MN'),
 (0.00042052451159490823, 'Portfolio_NSM (Freddie)'),
 (0.0004423537262058006, 'JumboVintage_2015'),
 (0.00045256651452697994, 'MaxTextP3to6mosProb_14d'),
 (0.0004692636621855568, 'PropState_WA'),
 (0.0005014617286076678, 'PropState_NC'),
 (0.0005053623169955102, 'MaxTextPIF3to6mosProb_7d'),
 (0.0005065301123047635, 'JumboVintage_2010'),
 (0.0005276869310660936, 'Num30p7to9MosAgo'),
 (0.0005476089858416531, 'NumPifQtLastMo'),
 (0.0005613565423625111, 'PropState_NH'),
 (0.0006259139559943932, 'ListedLast6Mos_ind'),
 (0.0006419244662016615

In [114]:
drop=[x[1] for x in ListImp if x[0]<1e-4]
Purchase_input_features=[x for x in Purchase_input_features if x not in drop]

In [115]:
len(Purchase_input_features)

152

In [ ]:
n_estimators=300
Learning Rate=0.05
Max_depth=5
0.8756079606698741
0.7333193642051067

n_estimators=750
Learning Rate=0.02
Max_depth=4
0.8268265110287352
0.7303259795851169

n_estimators=500
Learning Rate=0.03
Max_depth=4
0.8250519231274811
0.7327506211273086

In [118]:
with open('out/Purchase_input_features_woTU.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(Purchase_input_features)

In [ ]:
with open('out/Purchase_input_features.csv', newline='') as myfile:
    reader=csv.reader(myfile)
    input_features=list()
    for row in reader:
        input_features=input_features+row

In [119]:
joblib.dump(gbclf_P_NL, 'out/Purchase_GB_woTU.pkl')

['out/Purchase_GB_woTU.pkl']

In [103]:
Purchase_input_features=[x for x in Purchase_input_features if x not in TUData]

In [107]:
len(Purchase_input_features)

155

In [105]:
Purchase_input_features

['VacRt12moChg',
 'PropType_PUD',
 'Channel_Wholesale',
 'CurIntRate',
 'SFHS12moChg',
 'NumCTLast3Mos',
 'OrigCLTV',
 'SFHS6moChgPct',
 'PropState_NH',
 'PropType_SFH',
 'HHMedInc6moChgPct',
 'OrigVal',
 'HHMedInc6moChg',
 'UnempRt12moChgPct',
 'SFHS12moChgPct',
 'RICEMaxLast24mos',
 'JumboVintage_2011',
 'CurPropVal',
 'HStarts12moChgPct',
 'PropState_UT',
 'UPB',
 'HAI12moChgPct',
 'MOR12moChg',
 'SATO',
 'LFPR6moChg',
 'PropState_NY',
 'Division_ABS',
 'NumPifQtLast6Mos',
 'PropState_MI',
 'Zillow_nan',
 'PropState_TN',
 'Prod_FIXED',
 'PropState_OK',
 'LFPR6moChgPct',
 'PropType_MFH',
 'MaxWebPIF3to6mosProb_28d',
 'MOP12moChg',
 'MaxTextPIFP3to6mosProb_28d',
 'MOP6moChgPct',
 'Num30pLast12Mos',
 'UnempRt6moChg',
 'PropState_OH',
 'HStarts12moChg',
 'MaxWebPIF2mosProb_7d',
 'RICETotal7to24Mos',
 'MaxWebP3to6mosProb_14d',
 'VacRt6moChgPct',
 'MaxTextPIF2mosProb_28d',
 'MaxTextPIFP3to6mosProb_7d',
 'PropState_MN',
 'RICEMaxLast6mos',
 'OccType_Second Home',
 'HHolds12moChg',
 'OccTyp

In [13]:
pred_pif=gbclf_PIF_NL.predict_proba(df_pif_train_NL[PIF_input_features])
df_pif_train_NL['prob_2mos']=pred_pif[:,1]
df_pif_train_NL['prob_3to6mos']=pred_pif[:,2]
pred_p=gbclf_P_NL.predict_proba(df_Purch3to6_train_NL[Purchase_input_features])
df_Purch3to6_train_NL['prob_P_3to6mos']=pred_p[:,1]

pred_pif=gbclf_PIF_NL.predict_proba(df_pif_test_NL[PIF_input_features])
df_pif_test_NL['prob_2mos']=pred_pif[:,1]
df_pif_test_NL['prob_3to6mos']=pred_pif[:,2]
pred_p=gbclf_P_NL.predict_proba(df_Purch3to6_test_NL[Purchase_input_features])
df_Purch3to6_test_NL['prob_P_3to6mos']=pred_p[:,1]

In [22]:
gbclf_P_NL.get_params

<bound method BaseEstimator.get_params of GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.03, loss='exponential', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=789, subsample=0.5, verbose=1,
              warm_start=False)>

In [16]:
print('RMSE PIF 2mos')
print("Train "+str((mean_squared_error(df_pif_train_NL['prob_2mos'], df_pif_train_NL['PIF_2mos']))**0.5))
print("Test "+str((mean_squared_error(df_pif_test_NL['prob_2mos'], df_pif_test_NL['PIF_2mos']))**0.5))
print('RMSE PIF 3to6mos')
print("Train "+str((mean_squared_error(df_pif_train_NL['prob_3to6mos'], df_pif_train_NL['PIF_3to6mos']))**0.5))
print("Test "+str((mean_squared_error(df_pif_test_NL['prob_3to6mos'], df_pif_test_NL['PIF_3to6mos']))**0.5))

print('RMSE Purchase')
print("Train "+str((mean_squared_error(df_Purch3to6_train_NL['prob_P_3to6mos'], df_Purch3to6_train_NL['PIF_P_3to6mos']))**0.5))
print("Test "+str((mean_squared_error(df_Purch3to6_test_NL['prob_P_3to6mos'], df_Purch3to6_test_NL['PIF_P_3to6mos']))**0.5))

RMSE PIF 2mos
Train 0.09708316510263434
Test 0.10072808048103574
RMSE PIF 3to6mos
Train 0.18349626184742346
Test 0.18768209155589052
RMSE Purchase
Train 0.3813750615056086
Test 0.4420811349160028


In [20]:
print(ks(df_pif_train_NL, "prob_2mos", 'PIF_2mos'))
print(ks(df_pif_test_NL, "prob_2mos", 'PIF_2mos'))

print(ks(df_pif_train_NL, "prob_3to6mos", 'PIF_3to6mos'))
print(ks(df_pif_test_NL, "prob_3to6mos", 'PIF_3to6mos'))

print(ks(df_Purch3to6_train_NL, "prob_P_3to6mos", 'PIF_P_3to6mos'))
print(ks(df_Purch3to6_test_NL, "prob_P_3to6mos", 'PIF_P_3to6mos'))

58.74171700374487
55.96644893504666
29.583860174505265
25.05978375958684
59.2958278045166
40.301970878103056


## Random Forest

In [638]:
input_features=[
    'TotalLogins', 'NumPifQtLast12Mos', 'NumPifQtLast6Mos','NumPifQtLast3Mos', 'NumPifQtLastMo', 'AddrMismatch',
#     'TextPIF2mosProb', 'TextPIF3to6mosProb','TextPurchase3to6mosProb', 
#     'TextPIFPurchase3to6mosProb', 'WebPIF2mosProb', 'WebPIF3to6mosProb', 'WebPurchase3to6mosProb',
#     'WebPIFPurchase3to6mosProb', 
    'MaxTextPIF2mosProb_7d', 'MaxTextPIF2mosProb_14d', 'MaxTextPIF2mosProb_28d', 
    'MaxTextPIF3to6mosProb_7d', 'MaxTextPIF3to6mosProb_14d', 'MaxTextPIF3to6mosProb_28d', 'MaxTextP3to6mosProb_7d', 
    'MaxTextP3to6mosProb_14d', 'MaxTextP3to6mosProb_28d', 'MaxTextPIFP3to6mosProb_7d', 'MaxTextPIFP3to6mosProb_14d',
    'MaxTextPIFP3to6mosProb_28d', 'MaxWebPIF2mosProb_7d', 'MaxWebPIF2mosProb_14d', 'MaxWebPIF2mosProb_28d', 
    'MaxWebPIF3to6mosProb_7d', 'MaxWebPIF3to6mosProb_14d', 'MaxWebPIF3to6mosProb_28d', 'MaxWebP3to6mosProb_7d',
    'MaxWebP3to6mosProb_14d', 'MaxWebP3to6mosProb_28d', 'MaxWebPIFP3to6mosProb_7d', 'MaxWebPIFP3to6mosProb_14d',
    'MaxWebPIFP3to6mosProb_28d', 'TotalLogins_7d', 'TotalLogins_14d', 'TotalLogins_28d','NumPifQuote_7d',
    'NumPifQuote_14d', 'NumPifQuote_28d','Prod_ARM','Prod_FIXED','Prod_Ballon','Prod_ARM2yr','Prod_ARM3yr',
    'Prod_ARM5yr','Prod_IO1yr','Prod_IO2yr','Prod_IO3yr','Prod_IO5yr','Prod_IO7yr','Prod_IO10yr','ZillowListingScore',
    'OrigTermLong','OrigTermShort','CurIntRate','UPB','SATO','OrigVal','MTMCLTV','Num30pLast12Mos','LoanAge',
    'RIMaxLast6mos','RIMaxLast12mos','RIMaxLast24mos','RICEMaxLast6mos','RICEMaxLast12mos','RICEMaxLast24mos',
    'Num30p1to3MosAgo','Num30p4to6MosAgo','Num30p7to9MosAgo','Num30p10to12MosAgo',
    'RITotal7to24Mos','RICETotal7to24Mos','RIMax7to24mos','RICEMax7to24mos','CurPropVal', 'Zillow_nan'
]+list(dummies.columns)

In [620]:
###################
# No Zillow Score
###################

input_features.remove('ZillowListingScore')

# for depth in [5,6,7,8]:
# for samples in [100,300,500,1000]:
#     for n in [100,200,300,400,500]:
rfclf_P_NL=RandomForestClassifier(n_estimators=200, max_depth=8, min_samples_split=100,
                                    random_state=159, n_jobs=16, verbose=0)
# rfclf_PIF_NL=GradientBoostingClassifier(loss='deviance', n_estimators=100, max_depth=6, subsample=0.3, 
#                                         random_state=123, verbose=1, learning_rate=0.1)
rfclf_P_NL.fit(df_Purch3to6_train_NL[input_features],df_Purch3to6_train_NL['PIF_P_3to6mos'])
# print('Max_depth='+str(depth))
# print('n_estimators='+str(n))
# print('max_depth='+str(depth)+' min_samples_split='+str(samples))
print(rfclf_P_NL.score(df_Purch3to6_train_NL[input_features],df_Purch3to6_train_NL['PIF_P_3to6mos']))
print(rfclf_P_NL.score(df_Purch3to6_test_NL[input_features],df_Purch3to6_test_NL['PIF_P_3to6mos']))

ValueError: list.remove(x): x not in list

In [640]:
# for depth in [8,9,10]:
# # for samples in [100,300,500,1000]:
#     for n in [100,200,300]:
rfclf_P_NL=RandomForestClassifier(n_estimators=300, max_depth=9, min_samples_split=100,
                                    random_state=159, n_jobs=16, verbose=0)
# rfclf_PIF_NL=GradientBoostingClassifier(loss='deviance', n_estimators=100, max_depth=6, subsample=0.3, 
#                                         random_state=123, verbose=1, learning_rate=0.1)
rfclf_P_NL.fit(df_Purch3to6_train_NL[input_features],df_Purch3to6_train_NL['PIF_P_3to6mos'])
# print('Max_depth='+str(depth))
# print('n_estimators='+str(n))
# print('max_depth='+str(depth)+' min_samples_split='+str(samples))
print(rfclf_P_NL.score(df_Purch3to6_train_NL[input_features],df_Purch3to6_train_NL['PIF_P_3to6mos']))
print(rfclf_P_NL.score(df_Purch3to6_test_NL[input_features],df_Purch3to6_test_NL['PIF_P_3to6mos']))

0.7635068352480585
0.6848916376487708


In [674]:
# for depth in [3,4,5,6,7,8,9]:
#     for n in [300,500,700]:
#         for lr in [0.01,0.02,0.05,0.1]:
depth=3
n=300
lr=0.02
gbclf_P_NL=GradientBoostingClassifier(loss='deviance', n_estimators=n, max_depth=depth, subsample=0.5, 
                                        random_state=123, verbose=0, learning_rate=lr)
gbclf_P_NL.fit(df_Purch3to6_train_NL[input_features],df_Purch3to6_train_NL['PIF_P_3to6mos'])
print(' ')
print('Max_depth='+str(depth))
print('n_estimators='+str(n))
print('learning_rate='+str(lr))
# print('max_depth='+str(depth)+' min_samples_split='+str(samples))
print(gbclf_P_NL.score(df_Purch3to6_train_NL[input_features],df_Purch3to6_train_NL['PIF_P_3to6mos']))
print(gbclf_P_NL.score(df_Purch3to6_test_NL[input_features],df_Purch3to6_test_NL['PIF_P_3to6mos']))

 
Max_depth=3
n_estimators=300
learning_rate=0.02
0.7263461908734998
0.6926874902551843


In [641]:
df_Purch3to6_train_NL.shape

(77905, 860)

In [639]:
df_Purch3to6_train_NL=df[(df['sample']=='DEV') & (df['ActiveListing']==0) & (df['PIF_3to6mos']==1) & 
                         ((df['PURPOSE']=='Purchase') | (df['PURPOSE']=='Refi'))]
df_Purch3to6_test_NL=df[(df['sample']=='VAL') & (df['ActiveListing']==0) & (df['PIF_3to6mos']==1) & 
                       ((df['PURPOSE']=='Purchase') | (df['PURPOSE']=='Refi'))]

In [636]:
df_Purch3to6_train_NL.ZillowListingScore.describe()

count    77905.000000
mean         0.487284
std          0.107772
min          0.035000
25%          0.408000
50%          0.453000
75%          0.526000
max          0.949000
Name: ZillowListingScore, dtype: float64

In [609]:
df_Purch3to6_test_NL.shape

(19241, 858)

In [675]:
importances=zip(gbclf_P_NL.feature_importances_,input_features)
ListImp=list(importances)
sorted(ListImp)

[(4.588684831126462e-05, 'PropState_WV'),
 (7.331017253169858e-05, 'Prod_IO7yr'),
 (0.00011730433060668656, 'Portfolio_FNMA-NSTR'),
 (0.00015809226808493625, 'JumboVintage_2014'),
 (0.0001640327967601455, 'Channel_Bulk Acquisition'),
 (0.0002631477407313756, 'JumboVintage_2015'),
 (0.0002958412627503058, 'Portfolio_Aurora (Freddie)'),
 (0.00030281506786459785, 'Channel_Retail'),
 (0.0003243637772663132, 'Num30p10to12MosAgo'),
 (0.0003443432017144075, 'PropTypeEDW_Single Family Townhouse'),
 (0.0003646754959621376, 'PropState_NJ'),
 (0.00038814414465133376, 'PropState_NV'),
 (0.00043094186729223256, 'PropState_MA'),
 (0.00045392331048379965, 'PropState_SC'),
 (0.0004672744230733901, 'PropState_AR'),
 (0.00047117248257840704, 'MaxTextPIF3to6mosProb_28d'),
 (0.0004985122372933108, 'PropState_FL'),
 (0.0006136013805336115, 'Prod_ARM5yr'),
 (0.0006148398382101694, 'Portfolio_NSM (Government)'),
 (0.0006307846695624396, 'Channel_Builder'),
 (0.0006389657800966272, 'JumboVintage_<2010'),
 (0.

In [669]:
len(ListImp)

92

In [565]:
p=os.path.normpath('S:\Risk Management\Modeling\Projects\Marketing\PurchPropensity\B - Data Exploration\PIF_IVS_Pandas.csv')
df[['LoanNumber','AsOfDate']+output].to_csv(p,index=False)

In [673]:
drop=[x[1] for x in ListImp if x[0]<1e-6]
input_features=[x for x in input_features if x not in drop]

In [243]:
df_Purch3to6_test_NL.PIF_P_3to6mos.mean()

0.5603715170278638

In [676]:
pred=gbclf_P_NL.predict_proba(df_Purch3to6_test_NL[input_features])
df_Purch3to6_test_NL['prob_P_3to6mos']=pred[:,1]
score=df_Purch3to6_test_NL[['prob_P_3to6mos','PIF_P_3to6mos']].copy()
score.sort_values('prob_P_3to6mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_P_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

PIF_P_3to6mos                  
            count     sum      mean
bin                                
0            1924  1722.0  0.895010
1            1924  1592.0  0.827443
2            1924  1229.0  0.638773
3            1924  1261.0  0.655405
4            1924  1163.0  0.604470
5            1924   897.0  0.466216
6            1924   718.0  0.373181
7            1924   673.0  0.349792
8            1924   651.0  0.338358
9            1925   325.0  0.168831

In [466]:
pred=rfclf_P_NL.predict_proba(df_Purch3to6_test_NL[rfclf_P_NL_features])
df_Purch3to6_test_NL['prob_P_3to6mos']=pred[:,1]
score=df_Purch3to6_test_NL[['prob_P_3to6mos','PIF_P_3to6mos']].copy()
score.sort_values('prob_P_3to6mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_P_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

PIF_P_3to6mos                  
            count     sum      mean
bin                                
0            1924  1844.0  0.958420
1            1924  1474.0  0.766112
2            1924  1540.0  0.800416
3            1924  1197.0  0.622141
4            1924  1117.0  0.580561
5            1924   983.0  0.510915
6            1924   682.0  0.354470
7            1924   600.0  0.311850
8            1924   453.0  0.235447
9            1925   341.0  0.177143

In [405]:
with open('rfclf_P_NL_input_features.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(input_features)

In [406]:
joblib.dump(rfclf_P_NL, 'rfclf_P_NL.pkl')

['rfclf_P_NL.pkl']

# Unconditional Probability

In [10]:
df_test_NL=df[(df['sample']=='VAL') & (df['ActiveListing']==0) & 
        ((df['PIF_3to6mos']==0) | ((df['PIF_3to6mos']==1)&((df['PURPOSE']=='Purchase') | (df['PURPOSE']=='Refi'))))]

In [704]:
# rfclf_PIF_NL=joblib.load('rfclf_PIF_NL.pkl')
# rfclf_P_NL=joblib.load('rfclf_P_NL.pkl')

In [10]:
gbclf_PIF_NL=joblib.load('out/PIF_GB_woTU.pkl')
gbclf_P_NL=joblib.load('out/Purchase_GB_woTU.pkl')

# with open('rfclf_PIF_NL_input_features.csv',newline='') as csvfile:
#     reader=csv.reader(csvfile)
#     rfclf_PIF_NL_features=list()
#     for row in reader:
#         rfclf_PIF_NL_features=rfclf_PIF_NL_features+row

# with open('rfclf_P_NL_input_features.csv',newline='') as csvfile:
#     reader=csv.reader(csvfile)
#     rfclf_P_NL_features=list()
#     for row in reader:
#         rfclf_P_NL_features=rfclf_P_NL_features+row
        
with open('out/PIF_input_features_woTU.csv', newline='') as myfile:
    reader=csv.reader(myfile)
    PIF_input_features=list()
    for row in reader:
        PIF_input_features=PIF_input_features+row

with open('out/Purchase_input_features_woTU.csv', newline='') as myfile:
    reader=csv.reader(myfile)
    Purchase_input_features=list()
    for row in reader:
        Purchase_input_features=Purchase_input_features+row

In [96]:
df_test_NL.columns

Index(['LoanNumber', 'PropZipChar', 'PropState', 'PropTypeEDW', 'Units',
       'OccType', 'OrigDate', 'FirstPmtDate', 'MaturityDate', 'Lien',
       ...
       'm1lntyp_nan', 'DV_PIF', 'prob_2mos', 'prob_3to6mos', 'prob_P_3to6mos',
       'prob_PIFP_3to6mos', 'prob_PIFR_3to6mos', 'probDelta_PR3to6mos',
       'PIF_P_3to6mos_uncon', 'PIF_R_3to6mos_uncon'],
      dtype='object', length=974)

In [14]:
pred_pif=gbclf_PIF_NL.predict_proba(df_test_NL[PIF_input_features])
df_test_NL['prob_2mos']=pred_pif[:,1]
df_test_NL['prob_3to6mos']=pred_pif[:,2]
pred_p=gbclf_P_NL.predict_proba(df_test_NL[Purchase_input_features])
df_test_NL['prob_P_3to6mos']=pred_p[:,1]
df_test_NL['prob_PIFP_3to6mos']=df_test_NL['prob_3to6mos']*df_test_NL['prob_P_3to6mos']
df_test_NL['prob_PIFR_3to6mos']=df_test_NL['prob_3to6mos']*(1-df_test_NL['prob_P_3to6mos'])
df_test_NL['probDelta_PR3to6mos']=df_test_NL['prob_PIFP_3to6mos']-df_test_NL['prob_PIFR_3to6mos']

In [43]:
len(gbclf_PIF_NL.feature_importances_)

170

In [25]:
df_test_NL['PIF_P_3to6mos_uncon']=0
df_test_NL['PIF_P_3to6mos_uncon'][df_test_NL['PIF_P_3to6mos']==1]=1
df_test_NL['PIF_R_3to6mos_uncon']=0
df_test_NL['PIF_R_3to6mos_uncon'][(df_test_NL['PIF_3to6mos']==1) & (df_test_NL['PURPOSE']=='Refi')]=1

In [26]:
df_test_NL.sort_values(['LoanNumber','AsOfDate'], inplace=True)
score=df_test_NL.groupby('LoanNumber').last()
# if want LoanNumber to be a column instead of the index
# score=df_test_NL.groupby('LoanNumber').last().reset_index()

In [27]:
score[['AsOfDate']].shape

(85819, 1)

In [37]:
# score=(df_test_NL[['probDelta_PR3to6mos','PIF_P_3to6mos_uncon', 'PIF_R_3to6mos_uncon']]
# #        [df_test_NL['prob_3to6mos']>=0.05]
#        .copy())
score.sort_values('probDelta_PR3to6mos',ascending=False, inplace=True)
lift(score,50)
score[['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon','bin']].groupby('bin').agg(['count','sum','mean']).to_clipboard()

## Create Grid

In [37]:
def Binning(df,Bin,BinName):
    #df is sorted
    n=int(df.shape[0]/Bin)
    df[BinName]=0
    for i in range(Bin):
        a=int(i*n)
        b=int((i+1)*n)
        if i<Bin-1:
            df[BinName][a:b]=i
        else:
            df[BinName][a:]=i

In [38]:
df_test_NL.sort_values(['LoanNumber','AsOfDate'], inplace=True)
score=df_test_NL.groupby('LoanNumber').last()
score.sort_values('prob_PIFP_3to6mos',ascending=False, inplace=True)
Binning(score,20,'PurchBin')
score.sort_values('prob_PIFR_3to6mos',ascending=False, inplace=True)
Binning(score,20,'RefiBin')

In [39]:
# Grid 1

score['Group']=""
score['Group'][((score['PurchBin']==0) & (score['RefiBin']>=2)) |
               ((score['PurchBin']==1) & (score['RefiBin']>=4)) |
               ((score['PurchBin']==2) & (score['RefiBin']>=5)) |
               ((score['PurchBin']==3) & (score['RefiBin']>=8)) |
               ((score['PurchBin']==4) & (score['RefiBin']>=16)) |
               ((score['PurchBin']==5) & (score['RefiBin']>=18))]="A High Purchase Low Refi"
               

# df_Purch3to6_train_NL=df[(df['sample']=='DEV') & (df['ActiveListing']==0) & (df['PIF_3to6mos']==1) & 
#                          ((df['PURPOSE']=='Purchase') | (df['PURPOSE']=='Refi'))]

score['Group'][((score['PurchBin']>=5) & (score['RefiBin']==0)) |
               ((score['PurchBin']>=6) & (score['RefiBin']==1)) |
               ((score['PurchBin']>=10) & (score['RefiBin']==2)) |
               ((score['PurchBin']>=13) & (score['RefiBin'].isin([3,4,5])))
               ]="B Low Purchase High Refi"

score['Group'][(score['Group']=="") &
               ((score['PurchBin']>=14) | (score['RefiBin']>=14))
               ]="D Low Purchase Low Refi"
                
score['Group'][(score['Group']=="")]="C High Purchase High Refi"

In [131]:
# Grid 2

score['Group']=""
score['Group'][((score['PurchBin']==0) & (score['RefiBin']>=1)) |
               ((score['PurchBin']==1) & (score['RefiBin']>=4)) |
               ((score['PurchBin']==2) & (score['RefiBin']>=5)) |
               ((score['PurchBin']==3) & (score['RefiBin']>=7)) |
               ((score['PurchBin']==4) & (score['RefiBin']>=17)) |
               ((score['PurchBin'].isin([5,6,7])) & (score['RefiBin']>=17))
               ]="A High Purchase Low Refi"

score['Group'][((score['PurchBin']>=5) & (score['RefiBin']==0)) |
               ((score['PurchBin']>=6) & (score['RefiBin']==1)) |
               ((score['PurchBin']>=10) & (score['RefiBin']==2)) |
               ((score['PurchBin']>=13) & (score['RefiBin'].isin([3,4,5]))) |
               ((score['PurchBin']>=18) & (score['RefiBin'].isin([6,7,8,9,10])))
               ]="B Low Purchase High Refi"

score['Group'][(score['Group']=="") &
               ((score['PurchBin']>=14) | (score['RefiBin']>=14))
               ]="D Low Purchase Low Refi"

score['Group'][(score['Group']=="")]="C High Purchase High Refi"

In [51]:
score[['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon','Group']].groupby(['Group']).agg(['count','sum','mean']).to_clipboard()

PIF_P_3to6mos_uncon                PIF_R_3to6mos_uncon               
                      count  sum      mean               count  sum      mean
RefiBin                                                                      
0                      4290  141  0.032867                4290  168  0.039161
1                      4290   88  0.020513                4290  150  0.034965
2                      4290   94  0.021911                4290  118  0.027506
3                      4290   88  0.020513                4290  107  0.024942
4                      4290   76  0.017716                4290   92  0.021445
5                      4290   67  0.015618                4290   81  0.018881
6                      4290   55  0.012821                4290   64  0.014918
7                      4290   56  0.013054                4290   54  0.012587
8                      4290   63  0.014685                4290   58  0.013520
9                      4290   58  0.013520                4290   51  0.011888
10                     4290   64  0.014918                4290   53  0.012354
11                     4290   68  0.015851                4290   46  0.010723
12                     4290   54  0.012587                4290   36  0.008392
13                     4290   58  0.013520                4290   35  0.008159
14                     4290   72  0.016783                4290   28  0.006527
15                     4290   72  0.016783                4290   21  0.004895
16                     4290   96  0.022378                4290   19  0.004429
17                     4290  113  0.026340                4290   18  0.004196
18                     4290  131  0.030536                4290   16  0.003730
19                     4309  133  0.030866                4309   13  0.003017

In [46]:
score[['ZillowListingScore','CurIntRate','NumPifQuote_28d','MTMCLTV','AddrMismatch','Group']].groupby(['Group']).agg(['mean'])

,ZillowListingScore,CurIntRate,NumPifQuote_28d,MTMCLTV,AddrMismatch
,mean,mean,mean,mean,mean
Group,,,,,
A High Purchase Low Refi,0.505364,4.425690,0.081309,67.452562,0.708364
B Low Purchase High Refi,0.438596,4.952650,0.131088,52.989158,0.017250
C High Purchase High Refi,0.468586,4.327084,0.062765,63.091182,0.092253
D Low Purchase Low Refi,0.441937,4.285692,0.003114,67.676058,0.202918


In [47]:
score[['prob_3to6mos','prob_PIFP_3to6mos']].describe()

,prob_3to6mos,prob_PIFP_3to6mos
count,85819.000000,85819.000000
mean,0.038060,0.020992
std,0.034567,0.029084
min,0.001294,0.000222
25%,0.022691,0.008479
50%,0.030041,0.013732
75%,0.040649,0.022811
max,0.835462,0.717235


In [136]:
(score[score['probDelta_PR3to6mos']>=0.021401445518270844][['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon']]
 .agg(['count','sum','mean']).to_clipboard())

In [193]:
score['Group2']=""
score.sort_values('probDelta_PR3to6mos',ascending=False, inplace=True)
score[:17146]['Group2']="A High Purchase Low Refi"
score.sort_values('probDelta_PR3to6mos',ascending=True, inplace=True)
score[:16700]['Group2']="B Low Purchase High Refi"
score.sort_values('prob_3to6mos',ascending=False, inplace=True)
score[score['Group2']==""].iloc[:28459]['Group2']="C High Purchase High Refi"
score['Group2'][(score['Group2']=="") & (score['prob_3to6mos']>0.02454798221486146)]="C High Purchase High Refi"
score['Group2'][score['Group2']==""]="D Low Purchase Low Refi"

score[['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon','Group2']].groupby(['Group2']).agg(['count','sum','mean']).to_clipboard()

In [217]:
score['Group2']=""
score.sort_values('probDelta_PR3to6mos',ascending=False, inplace=True)
score[:17146]['Group2']="A High Purchase Low Refi"
score.sort_values('probDelta_PR3to6mos',ascending=True, inplace=True)
score[:16700]['Group2']="B Low Purchase High Refi"
score.sort_values('prob_3to6mos',ascending=False, inplace=True)

In [221]:
score[['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon','Group3']].groupby(['Group3']).agg(['count','sum','mean'])

PIF_P_3to6mos_uncon                 \
                                       count  sum      mean   
Group3                                                        
                                       51973  623  0.011987   
A High Purchase Low Refi               17146  841  0.049049   
B Low Purchase High Refi               16700  183  0.010958   

                         PIF_R_3to6mos_uncon                 
                                       count  sum      mean  
Group3                                                       
                                       51973  607  0.011679  
A High Purchase Low Refi               17146  145  0.008457  
B Low Purchase High Refi               16700  476  0.028503

In [31]:
score['Group2']=score['Group']
score.sort_values('probDelta_PR3to6mos',ascending=False, inplace=True)
score['Group2'][score['probDelta_PR3to6mos']>0.02268486209476113]="A High Purchase Low Refi"
score.sort_values('probDelta_PR3to6mos',ascending=True, inplace=True)
score['Group2'][score['probDelta_PR3to6mos']<-0.01487043629117555]="B Low Purchase High Refi"

In [33]:
score[['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon','Group']].groupby(['Group']).agg(['count','sum','mean'])

PIF_P_3to6mos_uncon                 \
                                        count  sum      mean   
Group                                                          
A High Purchase Low Refi                15029  737  0.049039   
B Low Purchase High Refi                13449  145  0.010781   
C High Purchase High Refi               28758  556  0.019334   
D Low Purchase Low Refi                 28583  209  0.007312   

                          PIF_R_3to6mos_uncon                 
                                        count  sum      mean  
Group                                                         
A High Purchase Low Refi                15029  123  0.008184  
B Low Purchase High Refi                13449  399  0.029668  
C High Purchase High Refi               28758  495  0.017213  
D Low Purchase Low Refi                 28583  211  0.007382

In [249]:
a=pd.DataFrame([[1,1,9],[2,1,9],[3,2,9],[4,2,9]],columns=['a','b','c'])

In [252]:
a['c'].iloc[0]=100

In [247]:
a['c'][a['b']==2].iloc[0]=100

In [253]:
print(a)

   a  b    c
0  1  1  100
1  2  1    9
2  3  2    9
3  4  2    9


In [220]:
score['Group3']=score['Group2']
score[score['Group2']==""][:20]['Group3']='111'

In [167]:
score[score['Group2']=="1111"]

,PropZipChar,PropState,PropTypeEDW,Units,OccType,OrigDate,FirstPmtDate,MaturityDate,Lien,OrigBal,...,prob_P_3to6mos,prob_PIFP_3to6mos,prob_PIFR_3to6mos,probDelta_PR3to6mos,PIF_P_3to6mos_uncon,PIF_R_3to6mos_uncon,PurchBin,RefiBin,Group,Group2
LoanNumber,,,,,,,,,,,,,,,,,,,,,


In [431]:
####No Zillow Score

pred_pif=rfclf_PIF_NL.predict_proba(df_test_NL[input_features])
df_test_NL['prob_2mos']=pred_pif[:,1]
df_test_NL['prob_3to6mos']=pred_pif[:,2]
pred_p=rfclf_P_NL.predict_proba(df_test_NL[input_features])
df_test_NL['prob_P_3to6mos']=pred_p[:,1]
df_test_NL['prob_PIFP_3to6mos']=df_test_NL['prob_3to6mos']*df_test_NL['prob_P_3to6mos']
df_test_NL['prob_PIFR_3to6mos']=df_test_NL['prob_3to6mos']*(1-df_test_NL['prob_P_3to6mos'])

In [432]:
score=df_test_NL[['prob_PIFP_3to6mos','PIF_P_3to6mos_uncon', 'PIF_R_3to6mos_uncon']].copy()
score.sort_values('prob_PIFP_3to6mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon','bin']].groupby('bin').agg(['count','sum','mean'])

PIF_P_3to6mos_uncon                 PIF_R_3to6mos_uncon                
                  count   sum      mean               count   sum      mean
bin                                                                        
0                 20582  7414  0.360218               20582  1213  0.058935
1                 20582  6266  0.304441               20582  1557  0.075649
2                 20582  5548  0.269556               20582  2563  0.124526
3                 20582  4799  0.233165               20582  3505  0.170294
4                 20582  3998  0.194247               20582  4243  0.206151
5                 20582  3767  0.183024               20582  3955  0.192158
6                 20582  2892  0.140511               20582  4809  0.233651
7                 20582  2586  0.125644               20582  5080  0.246818
8                 20582  1793  0.087115               20582  4279  0.207900
9                 20591   970  0.047108               20591  2412  0.117139

In [549]:
def Binning(df,Bin,BinName):
    #df is sorted
    n=int(df.shape[0]/Bin)
    df[BinName]=0
    for i in range(Bin):
        a=int(i*n)
        b=int((i+1)*n)
        if i<Bin-1:
            df[BinName][a:b]=i
        else:
            df[BinName][a:]=i

In [550]:
score=(df_test_NL[['prob_PIFP_3to6mos','prob_3to6mos','prob_P_3to6mos','PIF_P_3to6mos_uncon',
                   'PIF_R_3to6mos_uncon']]
#        [df_test_NL['prob_3to6mos']>=0.05]
       .copy())
score.sort_values('prob_3to6mos',ascending=False, inplace=True)
Binning(score,10,'PIFBin')
score.sort_values('prob_P_3to6mos',ascending=False, inplace=True)
Binning(score,10,'PurchBin')

In [551]:
score.head()

,prob_PIFP_3to6mos,prob_3to6mos,prob_P_3to6mos,PIF_P_3to6mos_uncon,PIF_R_3to6mos_uncon,PIFBin,PurchBin
39191,0.115955,0.126370,0.917583,0,0,0,0
39193,0.115879,0.126287,0.917583,0,0,0,0
39194,0.112563,0.122686,0.917491,0,0,0,0
39196,0.137634,0.150043,0.917302,0,0,0,0
39185,0.070373,0.076758,0.916814,0,0,0,0


In [552]:
pd.pivot_table(score,values='PIF_P_3to6mos_uncon',index=['PIFBin'],columns=['PurchBin'],
               aggfunc='mean', fill_value=0)

PurchBin,0,1,2,3,4,5,6,7,8,9
PIFBin,,,,,,,,,,
0,0.079552,0.069194,0.043474,0.035200,0.030352,0.034700,0.016570,0.007502,0.008359,0.002285
1,0.042341,0.039492,0.026325,0.023911,0.013780,0.015595,0.003196,0.013143,0.016970,0.002033
2,0.037263,0.023075,0.029120,0.026730,0.014647,0.012538,0.007107,0.005166,0.007930,0.005507
3,0.019506,0.023893,0.018847,0.014969,0.019187,0.005177,0.013130,0.015095,0.002466,0.006175
4,0.023135,0.025076,0.011886,0.016428,0.012780,0.009445,0.012492,0.006707,0.006047,0.009318
5,0.019453,0.019614,0.013155,0.014728,0.015892,0.008342,0.007951,0.005547,0.004016,0.007946
6,0.017376,0.017340,0.018813,0.008492,0.009922,0.008864,0.008929,0.009140,0.006682,0.001851
7,0.019983,0.022299,0.013412,0.029459,0.010304,0.011293,0.018764,0.011249,0.007578,0.003407
8,0.019057,0.022585,0.020823,0.011967,0.008657,0.006110,0.005471,0.006220,0.005031,0.002195


In [553]:
pd.pivot_table(score,values='PIF_R_3to6mos_uncon',index=['PIFBin'],columns=['PurchBin'],
               aggfunc='mean', fill_value=0)

PurchBin,0,1,2,3,4,5,6,7,8,9
PIFBin,,,,,,,,,,
0,0.002663,0.009187,0.025813,0.023062,0.023019,0.045216,0.047674,0.061128,0.055471,0.046523
1,0.002151,0.010439,0.011054,0.016969,0.013496,0.026276,0.055402,0.031919,0.037334,0.019110
2,0.001672,0.010104,0.009141,0.015593,0.016496,0.017594,0.024779,0.025294,0.044714,0.022888
3,0.003495,0.008424,0.018692,0.013842,0.010466,0.009678,0.022059,0.016131,0.028870,0.040932
4,0.004458,0.013416,0.007448,0.005586,0.017394,0.006746,0.016039,0.017520,0.028074,0.025837
5,0.002889,0.005411,0.015477,0.006941,0.015741,0.010921,0.023690,0.015442,0.023022,0.035624
6,0.007168,0.005320,0.001939,0.006108,0.016106,0.006615,0.009970,0.016332,0.021239,0.020178
7,0.002815,0.000000,0.000419,0.010113,0.004817,0.013057,0.011992,0.012440,0.017523,0.013229
8,0.000000,0.002907,0.001225,0.002459,0.005667,0.007050,0.012393,0.008249,0.015092,0.012327


In [554]:
pd.pivot_table(score,values='PIF_P_3to6mos_uncon',index=['PIFBin'],columns=['PurchBin'],
               aggfunc='count', fill_value=0)

PurchBin,0,1,2,3,4,5,6,7,8,9
PIFBin,,,,,,,,,,
0,9013,12082,11041,6591,4909,3804,3440,3599,2632,6126
1,8833,7951,8775,7779,7039,4681,4693,5326,3241,4919
2,8373,7324,7658,6734,7032,4945,5206,5614,4540,5811
3,8869,6529,6420,6213,5733,4443,5712,6757,6893,5668
4,8299,6261,6310,6087,5634,5188,6484,7306,6946,4722
5,5192,5914,5169,5907,6607,6593,6163,6670,7471,7551
6,4604,5075,5156,6712,6954,7559,6720,6674,8381,5402
7,3553,4036,4772,6823,7473,8501,7088,7556,8446,4989
8,3096,4472,4082,6100,6353,8511,8957,7395,8349,5922


In [594]:
# score[['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon','bin']].groupby('bin').agg(['count','sum','mean'])
ks(score,'prob_PIFP_3to6mos','PIF_P_3to6mos_uncon')

27.932148862830164

In [681]:
ks(score,'prob_PIFP_3to6mos','PIF_P_3to6mos_uncon')

27.514812531089117

In [52]:
score[PIF_input_features].mean().to_clipboard()

# Check Impact of Text and Web Scores

In [54]:
df_test_NL2=df_test_NL.copy()
df_test_NL2[['MaxTextPIF2mosProb_7d', 'MaxTextPIF2mosProb_14d', 'MaxTextPIF2mosProb_28d', 
    'MaxTextPIF3to6mosProb_7d', 'MaxTextPIF3to6mosProb_14d', 'MaxTextPIF3to6mosProb_28d', 'MaxTextP3to6mosProb_7d', 
    'MaxTextP3to6mosProb_14d', 'MaxTextP3to6mosProb_28d', 'MaxTextPIFP3to6mosProb_7d', 'MaxTextPIFP3to6mosProb_14d',
    'MaxTextPIFP3to6mosProb_28d', 'MaxWebPIF2mosProb_7d', 'MaxWebPIF2mosProb_14d', 'MaxWebPIF2mosProb_28d', 
    'MaxWebPIF3to6mosProb_7d', 'MaxWebPIF3to6mosProb_14d', 'MaxWebPIF3to6mosProb_28d', 'MaxWebP3to6mosProb_7d',
    'MaxWebP3to6mosProb_14d', 'MaxWebP3to6mosProb_28d', 'MaxWebPIFP3to6mosProb_7d', 'MaxWebPIFP3to6mosProb_14d',
    'MaxWebPIFP3to6mosProb_28d']]=0.0

In [55]:
pred_pif=gbclf_PIF_NL.predict_proba(df_test_NL2[PIF_input_features])
df_test_NL2['prob_2mos']=pred_pif[:,1]
df_test_NL2['prob_3to6mos']=pred_pif[:,2]
pred_p=gbclf_P_NL.predict_proba(df_test_NL2[Purchase_input_features])
df_test_NL2['prob_P_3to6mos']=pred_p[:,1]
df_test_NL2['prob_PIFP_3to6mos']=df_test_NL2['prob_3to6mos']*df_test_NL2['prob_P_3to6mos']
df_test_NL2['prob_PIFR_3to6mos']=df_test_NL2['prob_3to6mos']*(1-df_test_NL2['prob_P_3to6mos'])
df_test_NL2['probDelta_PR3to6mos']=df_test_NL2['prob_PIFP_3to6mos']-df_test_NL2['prob_PIFR_3to6mos']

In [62]:
df_test_NL2['PIF_P_3to6mos_uncon']=0
df_test_NL2['PIF_P_3to6mos_uncon'][df_test_NL2['PIF_P_3to6mos']==1]=1
df_test_NL2['PIF_R_3to6mos_uncon']=0
df_test_NL2['PIF_R_3to6mos_uncon'][(df_test_NL2['PIF_3to6mos']==1) & (df_test_NL2['PURPOSE']=='Refi')]=1
df_test_NL2.sort_values(['LoanNumber','AsOfDate'], inplace=True)
score=df_test_NL2.groupby('LoanNumber').last()
score.sort_values('probDelta_PR3to6mos',ascending=False, inplace=True)
lift(score,50)
score[['PIF_P_3to6mos_uncon','PIF_R_3to6mos_uncon','bin']].groupby('bin').agg(['count','sum','mean']).to_clipboard()